In [1]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gc
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback

In [2]:
def save_data_to_disk(data, filename):
    if os.path.isfile(filename):
        existing_df = pd.read_csv(filename)
        data = pd.concat([existing_df, data], ignore_index=True)
        os.remove(filename)
        data.to_csv(filename,index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

In [3]:
df = pd.read_excel('Naukri_Extract_20231116.xlsx')

In [4]:
def extract_and_calculate_average(salary_str):
    l = []
    salary_str = salary_str.replace('Less than','')
    
    if '/month' in salary_str:
        salary_str = salary_str.replace('/month','').replace(',','')
        return (float(salary_str)*12)/100000
    
    try:
        a = salary_str.split('-')
        if a[0] == a[1]:
            salary_str = a[0]
    except:
        None
        
    try:    
        l = list(salary_str)
        if 'L' in l:    
            ind = l.index('L')
            l = ''.join(l[:ind])
            l = l.strip()
            l = [i.replace(',','') for i in l]
            l = ''.join(l[:ind])
            if '-' in l:    
                l = l.split('-')
                l = [float(i)/100000 if len(i) > 4 else float(i) for i in l]
                l = sum(l)/2
                return l
            else:
                print(l,'****',salary_str,end='****')
                l = (float(l)/100000 if len(l) > 4 else float(l))
                print(l)
                return l
        
        elif 'C' in l:
            ind = l.index('C')
            l = ''.join(l[:ind])
            l = l.strip()
            l = [i.replace(',','') for i in l]
            l = ''.join(l[:ind])
            if '-' in l:    
                l = l.split('-')
                l = [float(i)*100 if len(i) > 4 else float(i) for i in l]
                l = sum(l)/2
                return l
            else:
                l = (float(l)/100 if len(l) > 4 else float(l))
                return l[0]*100
        
        elif 'P' in l:
            ind = l.index('P')
            l = ''.join(l[:ind])
            l = l.strip()
            l = [i.replace(',','') for i in l]
            l = ''.join(l[:ind])
            if '-' in l:
                l = l.split('-')
                l = [float(i)/100000 if len(i) > 4 else float(i) for i in l]
                l = sum(l)/2
                return l 
            else:
                l = (float(l)/100000 if len(l) > 4 else float(l))
                return l 
    except:
        return salary_str
df['saledit'] = df['salary'].apply(extract_and_calculate_average)


30 **** 30 Lacs PA****30.0
30 **** 30 Lacs PA****30.0
12 **** 12 Lacs PA****12.0
15 **** 15 Lacs PA****15.0
1.75 **** 1.75 Lacs PA****1.75
2.5 **** 2.5 Lacs PA****2.5
1.5 **** 1.5 Lacs PA****1.5
45 **** 45 Lacs-1 Cr PA****45.0
25 **** 25 Lacs PA****25.0
18 **** 18 Lacs PA****18.0
1.5 **** 1.5 Lacs PA****1.5
25 **** 25 Lacs PA****25.0
2.25 **** 2.25 Lacs PA****2.25
2.25 **** 2.25 Lacs PA****2.25
70 **** 70 Lacs-1.25 Cr PA****70.0
8 **** 8 Lacs PA****8.0
9 **** 9 Lacs PA****9.0
20 **** 20 Lacs PA****20.0
19 **** 19 Lacs PA****19.0
22.5 **** 22.5 Lacs PA****22.5
22.5 **** 22.5 Lacs PA****22.5
2.75 **** 2.75 Lacs PA****2.75
2.75 **** 2.75 Lacs PA****2.75
17 **** 17 Lacs PA****17.0
30 **** 30 Lacs PA****30.0
32.5 **** 32.5 Lacs PA****32.5
25 **** 25 Lacs PA****25.0
2.25 **** 2.25 Lacs PA****2.25
15 **** 15 Lacs PA****15.0
25 **** 25 Lacs PA****25.0
2.5 **** 2.5 Lacs PA****2.5
2 **** 2 Lacs PA****2.0
1.5 **** 1.5 Lacs PA****1.5
75 **** 75 Lacs-1 Cr PA****75.0
4.5 **** 4.5 Lacs PA****4.5
4 **

In [5]:
ck = ['saledit','salary']

In [6]:
df[df['saledit'].apply(lambda x: isinstance(x, str))][ck]

,saledit,salary
1402,1 Cr PA,1 Cr PA
1410,9.5 Cr and above PA,9.5 Cr and above PA
37702,1 Cr PA,1 Cr PA


In [7]:
df['saledit'] = df['saledit'].fillna(0)

In [8]:
df = df[~df['saledit'].apply(lambda x: isinstance(x, str))]
df = df[~df['saledit'].apply(lambda x: x>100)]

In [9]:
df[df['saledit'].isna()][ck]

,saledit,salary


# Salary done

In [10]:
lidf = pd.read_excel('Linkedin_cleaned_latest_mapped_cols.xlsx')

In [11]:
sdf = pd.DataFrame()
org_cols = ([col for col in lidf.columns if 'org' in col])
sdf['all_org_cols'] = lidf[org_cols].apply(lambda row: '$&'.join(row.dropna().astype(str)), axis=1)
sdf['all_org_cols']

companies = []
for i in sdf['all_org_cols']:
    for j in i.split('$&'):
        companies.append(j)
print(len(companies))

17870


In [12]:
companiesdf = pd.DataFrame(companies,columns=['comps'])
companiesdf.drop_duplicates(inplace=True)
companiesdf.reset_index(drop = True, inplace=True)

In [13]:
companiesdf['comps'] = companiesdf['comps'].str.lower()
companiesdf['comps'] = companiesdf['comps'].str.replace(' · self-employed','').str.replace('· part-time','').str.replace('· freelance','')
companiesdf['comps'] = companiesdf['comps'].str.replace(' · full-time','').str.replace('  ',' ').str.replace('india','').str.replace('chennai','').str.replace('· internship','')
companiesdf['comps'] = companiesdf['comps'].str.strip()
companiesdf.drop_duplicates(inplace=True)
companiesdf.reset_index(drop = True, inplace=True)
companiesdf

,comps
0,swift infos
1,vidai labs
2,cavinkare
3,suguna group
4,apnaklub
...,...
12893,blockchain
12894,360 deg. ebusiness co.
12895,sigma construction company pvt. ltd
12896,jasper group


In [14]:
df['company_name'] = df['company_name'].str.lower()

In [15]:
len(df['company_name'].unique())

14212

In [16]:
# import pandas as pd
# from fuzzywuzzy import fuzz
# from multiprocessing import Pool
# from tqdm import tqdm  # Import tqdm for the progress bar

# # Function to calculate similarity scores and store them in a dictionary
# def calculate_similarity_scores(value, name_list, threshold=95):
#     best_score = 0
#     best_name = None

#     for name in name_list:
#         total_score = 0
#         for method in ['partial_ratio', 'token_set_ratio']:
#             similarity_score = getattr(fuzz, method)(value, name)
#             if similarity_score >= threshold:
#                 total_score += similarity_score

#         if total_score > best_score:
#             best_score = total_score
#             best_name = name

#     return best_name

# df['company_name'] = df['company_name'].astype(str)

# prl = companiesdf['comps'].tolist()

# tqdm.pandas()
# pool = Pool(processes=8)  # You can specify the number of processes here

# threshold = 95
# df['similarity_scores'] = df['company_name'].progress_apply(calculate_similarity_scores, args=(prl, threshold))

# pool.close()
# pool.join()

In [17]:
# df.to_excel('salary_and_company_mapping.xlsx',index= False)

In [18]:
# df = pd.read_excel('salary_and_company_mapping.xlsx')

In [19]:
# 'ratio'
# 'partial_ratio'
# 'token_set_ratio'
# 'token_sort_ratio'
# 'WRatio'
# 'SequenceMatcher'

In [20]:
df.rename(columns={'saledit':'mapped_average_sal'},inplace=True)
# df.rename(columns={'similarity_scores':'mapped_company_name'},inplace=True)

In [21]:
df

,job_title,company_name,experience,salary,location,description,posted_on,current_date,skill1,skill2,...,skill6,skill7,skill8,role,industry type,department,employment type,role category,job_description,mapped_average_sal
0,Aws Devops Engineer,tech mahindra,5-8 Yrs,16-22.5 Lacs PA,"Hybrid - Hyderabad/ Secunderabad, Telangana, B...","As a key member of the team, ensure success in...",1 Day Ago,2023-08-22,Terraform,devops,...,AWS Devops,Kubernetes,Nan,Nan,Nan,Nan,Nan,Nan,Nan,19.25
1,Hiring B. Tech Freshers || Hyderabad || Excell...,hcltech,0-0 Yrs,"50,000-2.5 Lacs PA","Hyderabad/ Secunderabad, Telangana",Profile: Technical troubleshooting Responsibil...,1 Day Ago,2023-08-22,Excellent Communication,HTML,...,Python,Technical analysis,Communication skills,Nan,Nan,Nan,Nan,Nan,Nan,1.5
2,Front End Engineer/ Front End Engineer Lead - ...,tech platform,6-11 Yrs,30 Lacs PA,"Bangalore/ Bengaluru, Karnataka",Demonstrated experience developing high-traffi...,1 Day Ago,2023-08-22,nextjs,javascript,...,css,web application,express,Nan,Nan,Nan,Nan,Nan,Nan,30.0
3,Day shift - Hindi Voice Process - 3L CTC,oxilum technologies,0-3 Yrs,2.5-3 Lacs PA,Bangalore/Bengaluru,Shift . Day Shift & Rotational Week OFF Qualif...,1 Day Ago,2023-08-22,BPO,Hindi Voice Process,...,No broker,Customer Service,Altruist,Nan,Nan,Nan,Nan,Nan,Nan,2.75
4,Great Salaries For Tech Support in Bangalore -...,jobshop,4-9 Yrs,4-5.5 Lacs PA,Bangalore/Bengaluru,This is a 24 / 7 support mostly evening and ni...,Just Now,2023-08-22,storyboarding,dreamweaver,...,instructional design,sound forge,captivate,Nan,Nan,Nan,Nan,Nan,Nan,4.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52751,Salesforce Developer,kdm dotsoft developers,2-4 Yrs,6.5-10 Lacs PA,Remote,Create Lightning components to build modern an...,1 Day Ago,2023-11-15,Salesforce Sales Cloud,Service Cloud,...,Cloud,Development,Sales force development,back end developer,it services & consulting,engineering - software & qa,"full time, freelance/homebased",software development,Role & responsibilities\nCustomization and Con...,8.25
52752,Ui Ux Developer,kdm dotsoft developers,2-6 Yrs,6.5-10 Lacs PA,Remote,User Experience (UX) Research: . Use this data...,1 Day Ago,2023-11-15,Ui Ux Development,UX,...,UI/UX,NaN,NaN,ui / ux designer,it services & consulting,"ux, design & architecture","full time, freelance/homebased",ui / ux,Role & responsibilities\nUser Interface (UI) D...,8.25
52753,Software Engineer And Developer,kdm dotsoft developers,2-5 Yrs,6.5-10 Lacs PA,Remote,Role & responsibilities Software Development:E...,1 Day Ago,2023-11-15,Software Development,Software Services,...,Software,NaN,NaN,software developer in test (sdet),it services & consulting,engineering - software & qa,"full time, freelance/homebased",quality assurance and testing,Role & responsibilities\nSoftware Development:...,8.25
52754,Business Intelligence Developer,kdm dotsoft developers,2-5 Yrs,6.5-10 Lacs PA,Remote,Role & responsibilities Data Modeling and Data...,1 Day Ago,2023-11-15,Business Intelligence Reporting,Business Intelligence,...,Intelligence,Development,NaN,business intelligence & analytics - other,it services & consulting,data science & analytics,"full time, freelance/homebased",business intelligence & analytics,Role & responsibilities\nData Modeling and Dat...,8.25


In [22]:
df.to_excel('salary_and_company_mapping.xlsx',index= False)

# company name done

In [23]:
df['avg_experience'] = df['experience'].str.replace('Yrs','')
df['avg_experience'] = df['avg_experience'].str.split('-')

In [24]:
def to_avg_exp(value):
    if isinstance(value, list) and len(value) >= 2:
        value[0] = int(value[0])
        value[1] = int(value[1])
        return sum(value) / 2
    else:
        # Handle cases where value is not a valid list
        return value

df['avg_experience'] = df['avg_experience'].apply(to_avg_exp)
df['avg_experience']

0        6.5
1        0.0
2        8.5
3        1.5
4        6.5
        ... 
52751    3.0
52752    4.0
52753    3.5
52754    3.5
52755    4.5
Name: avg_experience, Length: 52735, dtype: object

# average age done

In [25]:
# def replacefun(value):
#     fun_l = ['ing','ca','ge','ltd.',')','is','private','the','ari','stem','-','fi','karnataka','bangalore','.com','one','h','ies','.','a','bengaluru, karnataka',
#              'business','actor','dec','pvt ltd','care']
#     if value in fun_l:
#         return None
#     else:
#         return value
# df['mapped_company_name'] = df['mapped_company_name'].apply(replacefun)

In [26]:
# for k,v in df['mapped_company_name'].value_counts().items():
#     print(k,v)

In [27]:
# len(df['mapped_company_name'].unique()) # 1855

In [28]:
# df.to_excel('salary_and_company_mapping.xlsx',index= False)

In [29]:
# df['mapped_company_name'].notnull().sum()

In [30]:
# df['company_name'].notnull().sum()

# mapping instute and size.

In [31]:
([col for col in lidf.columns if 'emp' in col])

['company_0_emp_count',
 'company_1_emp_count',
 'company_2_emp_count',
 'company_3_emp_count',
 'company_4_emp_count',
 'company_5_emp_count',
 'company_6_emp_count',
 'company_7_emp_count',
 'company_8_emp_count',
 'company_9_emp_count',
 'company_10_emp_count',
 'company_11_emp_count',
 'company_12_emp_count',
 'company_13_emp_count',
 'company_14_emp_count',
 'company_15_emp_count',
 'company_16_emp_count',
 'company_17_emp_count',
 'company_18_emp_count',
 'company_19_emp_count',
 'employee_count',
 'mapped_company_0_emp_count',
 'mapped_company_1_emp_count',
 'mapped_company_2_emp_count',
 'mapped_company_3_emp_count',
 'mapped_company_4_emp_count',
 'mapped_company_5_emp_count',
 'mapped_company_6_emp_count',
 'mapped_company_7_emp_count',
 'mapped_company_8_emp_count',
 'mapped_company_9_emp_count',
 'mapped_company_10_emp_count',
 'mapped_company_11_emp_count',
 'mapped_company_12_emp_count',
 'mapped_company_13_emp_count',
 'mapped_company_14_emp_count',
 'mapped_company_15_e

In [32]:
org_cols

['org_0',
 'org_0_sub_0',
 'org_0_sub_1',
 'org_0_sub_2',
 'org_0_sub_3',
 'org_0_sub_4',
 'org_0_sub_5',
 'org_0_sub_6',
 'org_0_sub_7',
 'org_0_sub_8',
 'org_0_sub_9',
 'org_1',
 'org_1_sub_0',
 'org_1_sub_1',
 'org_1_sub_2',
 'org_1_sub_3',
 'org_1_sub_4',
 'org_1_sub_5',
 'org_1_sub_6',
 'org_1_sub_7',
 'org_1_sub_8',
 'org_2',
 'org_2_sub_0',
 'org_2_sub_1',
 'org_2_sub_2',
 'org_2_sub_3',
 'org_2_sub_4',
 'org_2_sub_5',
 'org_2_sub_6',
 'org_2_sub_7',
 'org_2_sub_8',
 'org_2_sub_9',
 'org_3',
 'org_3_sub_0',
 'org_3_sub_1',
 'org_3_sub_2',
 'org_3_sub_3',
 'org_3_sub_4',
 'org_3_sub_5',
 'org_3_sub_6',
 'org_3_sub_7',
 'org_3_sub_8',
 'org_3_sub_9',
 'org_4',
 'org_4_sub_0',
 'org_4_sub_1',
 'org_4_sub_2',
 'org_4_sub_3',
 'org_4_sub_4',
 'org_4_sub_5',
 'org_4_sub_6',
 'org_4_sub_7',
 'org_5',
 'org_5_sub_0',
 'org_5_sub_1',
 'org_5_sub_2',
 'org_5_sub_3',
 'org_6',
 'org_6_sub_0',
 'org_6_sub_1',
 'org_6_sub_2',
 'org_6_sub_3',
 'org_6_sub_4',
 'org_6_sub_5',
 'org_7',
 'org_7_

In [33]:
org_cols = ([col for col in lidf.columns if ('org' in col) and ('industry' not in col)])
orglen = int(org_cols[-1][4:6])

def splitjoin(value):
    value = value.split(',')
    if value[0] != '':
        return value[0]
    elif value[1] != '':
        return value[1]
    elif (value[0] == '') and (value[1] == ''):
        return None 

editdf=pd.DataFrame()
for i in range(orglen+1):
    subcol = [col for col in lidf.columns if 'org_'+str(i)+'_' in col]
    llis1 = lidf['org_'+str(i)]
    llis2 = lidf[subcol].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)
    
    llis1 = llis1.fillna('').astype(str)
    llis2 = llis2.fillna('').astype(str)
    
    editdf['org_'+str(i)] = llis1+','+llis2
    editdf['org_'+str(i)] = editdf['org_'+str(i)].apply(splitjoin)

In [34]:
for col in editdf:
    editdf[col] = editdf[col].str.lower()
    editdf[col] = editdf[col].str.replace(' · self-employed','').str.replace('· part-time','').str.replace('· freelance','')
    editdf[col] = editdf[col].str.replace(' · full-time','').str.replace('  ',' ').str.replace('india','').str.replace('chennai','').str.replace('· internship','')
    editdf[col] = editdf[col].str.strip()


In [35]:
# df['mapped_company_name']

In [36]:
for col in editdf:
    print(col.split('_')[1])
lidf.columns

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


Index(['name', 'location', 'org_0', 'title_0', 'job_0_duration',
       'job_0_location', 'org_0_sub_0', 'title_0_sub_0',
       'job_0_sub_0_duration', 'job_0_sub_0_location',
       ...
       'mapped_institute_0', 'mapped_institute_1', 'mapped_institute_2',
       'mapped_institute_3', 'mapped_institute_4', 'mapped_digree_0',
       'mapped_digree_1', 'mapped_digree_2', 'mapped_digree_3',
       'mapped_digree_4'],
      dtype='object', length=699)

In [37]:
# # def map_companies(ind, col):
# #     value = editdf[col][ind]
# #     colno = col.split('_')[1]
# #     if value == None:
# #         return
# #     prl = df['mapped_company_name'].tolist()
# #     for i in range(len(prl)):
# #         if prl[i] == value:
# #             df['comp_industry'][i] = lidf['company_'+'colno'+'_industry'][ind]
# # for col in editdf:
# #     for ind in range(len(editdf[col])):
# #         map_companies(ind, col)
    
# def map_companies(row):
#     index = row.name
#     for col in editdf.columns:
#         value = row[col]
#         if value is not None:
#             colno = col.split('_')[1]
            
#             if 'comp_industry' not in df.columns:
#                 df['comp_industry'] = None
            
#             if 'comp_industry' not in df.columns:
#                 df['comp_emp_count'] = None
            
#             matching_indices = df.index[df['mapped_company_name'] == value]
#             for idx in matching_indices:
#                 df.at[idx, 'comp_industry'] = lidf.at[index, f'company_{colno}_industry']
#                 df.at[idx, 'comp_emp_count'] = lidf.at[index, f'company_{colno}_emp_count']
# # Apply the mapping function to each row in editdf
# editdf.apply(map_companies, axis=1)

In [38]:
df.to_excel('salary_and_company_mapping.xlsx',index = False)

In [39]:
df['location'] = df['location'].str.lower().str.replace('hybrid -','').str.strip()

In [40]:
df = pd.read_excel('salary_and_company_mapping.xlsx')

In [41]:
df['mapped_job_title'] = df['job_title'].str.replace("[^a-zA-Z0-9]",' ',regex=True).str.replace('   ',' ').str.replace('  ',' ')
df['mapped_job_title'] = df['mapped_job_title'].str.lower().str.strip()

replace_lis = ['hiring b. tech freshers','hiring','b tech','freshers','hyderabad','excellent','comms','night', 'shifts','shift','day','3l ctc','ctc',
               'great', 'salaries', 'for', 'in', 'bangalore','is','tech mahindra','chennai','bca','m tech','cts','0','1','yrs','exp','urgent','cognizant',
               'graduates','joinees','immediate','cse','bulk','opening','fresher','mega','int l','sal', '4 2lpa','telugu','opportunity','going on','2 am',
               'itgc', 'big4','walk','drive','us','healthcare','ith','6','8','years','experience','with','3','6years','cust','cus','jobs','b sc','m sc',
               'mphil','biology','botany','zoology','gra','hybrid', 'remote', 'puc','grad','uk','hcl tech','looking','the leading tech firm','7lpa','tamil',
               'kannada','30 bpo','experienced','up to','btm', 'hebballocation','7','5lks','300','positions','direct','top','bpos','big','payment','company',
               'limited','pan','india','kr','malayalam','secunderabad','location','at manyata tech park','only experienced','blr','job opportunities','ca one tech',
               'infosys','payroll','ca one tech cloud','infosys','payroll','ca one tech cloud','openings','work from home','salary','13k','to','16k','amazon',
               'i','hdfc bank','the','leading telephone','brand','leading','work from office','noida','grads','share cv','upto 60','lpa','upto','sun technologies',
               'churchgate','mumbai','99acres','notice period','should be','30 45 days','30 days','45 days','one of mnc s','4 yr','mncs','wfh','wfo','iam',
               'hindi','english','joiners','less than','or less than','b e','mech','eee','required','marketing department','manyata tech','male','candidates','only',
               'job','ug','pg','pharm','lifescience','be','biotech','biomedi','work form home','call','sameera','18th aug 21st aug','5lpa','mnc','build your career',
               '4','4 2 l','uppal hafeezpeth','6lpa','7lks','8lpa','pune','gurgaon','50 old product','banaglore','hederabad','delhi','l id a02ub','secundarabad','imm',
               'joiner','blore','2pm','11pm','female','bengaluru','total rewards','rewards','iit','nit','bits','tier','walkin','interview','at','copart',
               'working from home','french','language','we are back','and','this','time','huge numbers','amazing','a','reputed organization','wroots','urgently l','urgently',
               '29k','immediately','12 months','5l','both','now','2years id a24mb','reputed','pharma industry','io pvt ltd','50 start ups','interviews','23rd aug till 25th aug',
               '27th aug 23','any','hyd','exp0 2','two way','cab 5','rotational','5l','virtual','25 aug 23','5 wrk','kpo','ahmedabad','copart','on','monday','28th','tuesday','29th',
               'eugia','shameerpet','spot','offer','ahmedabad','l2','dayshift','permanent','40k','whitefield','months','contract','role','ic','min of','home','working','mba','h r',
               'wanted','15','23','passout','mlt','operation','theatre','tech dialysis','dialysis','dialysis tech','24th','25th','aug','wanted15','23','nutrition','dietetics','dietitian',
               'nutritionist','food tech','tech guru','it open source','tech writer','journalist','all','wanted','x ray','technology mlt','zoom','19th','1st','sep',
               'kolkata','18','nanotech','operation theater','tech veterinary','sci','part ot','full','ed tech firm','anaesthesia','operation','radiology','perfusion tech','goregaon',
               '62','start your career','navi','belapur','dialysis','get','of','22k','hand','banking process','30k','50k','plus inc','20 21','one of s','an','travel','mca graduate',
               'mca','malad','name','u k based','m n c','5 days','days','work','2yrs','continental','2 continental','continental','hospital','gachibowli','generali','begumpet','subk',
               'females','27k','graduate','blended','centers','yr','req 28','req','italian experts','tech m 45','b19 sec opp dme college','e commerce','theater','scie','golden','opportunities',
               'nanotechnology','science','ii fashion','ii','hirings','hirinf','hire','hired','30th till','based','german','expert','chat process','b1 certified','australian'
               'sec 63','hr anjali','wipro','50 start ups','one s','kafka knowledge','40 k','32 k','mohali','chandigarh','co jaipur','ppc calls','swati','bfsi','chandivali','jaipur',
               'techmahindra','australian','ncr','k','month','apply','new','sat','sun','fix','off','bba','hinduja tech','kctc','dyson', 'norton','lalit','fresh','diploma','mechanical',
               'we','are','lalit','s','th','sept','en','btech','or','powai','rohtak','domestic','alisha','asap','stop', 'have', 'look','lms','oct','benefit','tamilnadu','city',
               'japan','mahipalpur',' bhosari','regional','joining','hcltech','open','position','b sec','sec','opp','dme','college','min','max','sohna','gujarat','l l','a',r'\d+k',
               '50','must 3l',r'\d+k','5 wrk','07','60','suyati technologies','kochi','iga','requirement','interested',r'\b(\w+)\b(?=.*\b\1\b)',r'^\b(?:coding|coding|coder|counselors|lead)\b$','nttdata',
               r'\b(\d+)\s*ctc\b',r'\b\d+[myMY]\b','aakash','byju','was','punjab','eia',r'\b(?:\d+\s*[lL][pP][aA]|2\s*l)\b','magnasoft',r'^\b(?:tech|process|candidate|better|associate|support|trainer|collections|medical)\b$','airoli',
               'way trasportation','portuguese','corporates gifting co','dailyrounds','domlur','manchar rajguru nagar road','datamatics andheri',r'^\b(?:internship|executive|local seo|international|intern|bpo|head)\b$',
               r'\b\d+\s\d{2}am\b',r'\bl pa\b','lucknow','hcl','thursday','engineer up','dubai','gurugram','research fellow phd scholar','retirement pension','safeai','popup calls','matrimony','com','9'
               r'\b\d+(?:st|nd|rd|th)\b','reminder','calling mandatory','strong coder','biotechnology engineers','biotechnology','genetic engineer', 'molecular','wa','em ip op',
               'perfusion','cardiac tech','physician','assistant','gradua','ophthalmic','nursing assistant','clinical care tech','b m','d pharma','Biotechnology engineer','biomedical engineer','biomed','bioscienc',
               'biotech Engineers','bvsc',r'\b\d+-\d+\b','biomedical engineer gr','2023','non technical','b m','engineers','22','gr','biomedical','bioinformatics','pharmacy','physiotherapy','nursing',
               'b m d','gradu','micro','microbiology','paramedical','bsc','msc','inorganic','organic chemistry','trauma','technology','neuro','want','vacancies','gnm','staff nurse','dgnm',
               'passoutbsc','dmlt','medical lab technician','bds','bhms','bams','bnys','bums','bsms','dentist','microbiologist','x','nurse','eligibility','verification','end rcm','biochemistry',r'\b[A-Za-z]\b',
               'physiotherapist','bpt','cell','recruiting'] 

pattern = r'\b(?:' + '|'.join(replace_lis) + r')\b'

df['mapped_job_title'] = df['mapped_job_title'].str.replace(pattern, '', regex=True)
df['mapped_job_title'] = df['mapped_job_title'].str.replace(r'\s+', ' ', regex=True).str.strip().str.lower()
df['mapped_job_title'] = df['mapped_job_title'].str.replace(pattern, '', regex=True)
df['mapped_job_title'] = df['mapped_job_title'].str.replace(r'\s+', ' ', regex=True).str.strip().str.lower()
# for job_title, mapped_job_title in zip(df['job_title'], df['mapped_job_title']):
#     print(f'Job Title: {job_title}, Mapped Job Title: {mapped_job_title}')

In [42]:
df['mapped_job_title_1'] = df['mapped_job_title']

In [43]:
df['mapped_job_title_1'] = df['mapped_job_title_1'].str.title() 

In [44]:
titles = df['mapped_job_title'].tolist()

In [45]:
from collections import defaultdict
title_counts = defaultdict(int)

for title in titles:
    title_counts[title] += 1

# Step 3: Sort the names based on their counts
sorted_title = sorted(title_counts.keys(), key=lambda title: title_counts[title], reverse=True)
sorted_title

titlec = []
# Print the sorted names
for title in sorted_title:
    titlec.append(f"{title}: {title_counts[title]}")


In [46]:
titledict = {}

In [47]:
skilcol = ([col for col in df.columns if 'skil' in col])

In [48]:
unique_roles = list(df['role'].unique())
unique_roles.remove('Nan')
unique_data_list = []

for role in unique_roles:
    unique_data = {}
    unique_data['role'] = role 
    unique_data['no_of_data'] = df[df['role'] == role].shape[0]  # Count of rows for the role
    try:
        commastr =  ','.join(df[df['role'] == role][skilcol].dropna().values.tolist()[0])  # Concatenate skills into a comma-separated string
    except:
        commastr = None
    unique_data['skills'] = commastr
    unique_data_list.append(unique_data)

# Create a DataFrame from the list of unique data
unique_datadf = pd.DataFrame(unique_data_list)
unique_datadf = unique_datadf.sort_values('no_of_data',ascending=False)
unique_datadf.to_excel('unique_roles_and_skills_from_naukri_latest.xlsx',index=False)

In [49]:
df[df['role'] != 'Nan'].shape

(12538, 26)

In [50]:
anyinlist = ['mis','management','lead','tech','executive','manager','head']
allinlist = ['mis|management|lead|manager|executive|head']
notinlist = ['admission','hemis','missi','soft','customer','voice']

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]
filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
                df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

apl = set(filtered_df['job_title'].str.lower())
titledict['Lead/Management'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['consultant']
# allinlist = ['consultant']
# notinlist = []# founder
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['consultant'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['support','tech','troubleshooting','engineer','lead','engineer','specialist','system','fluent','language']
# allinlist = []
# notinlist = ['voice','chat','customer','cust','production','analyst','devops']# founder
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['tech support'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['support','care','customer','executive','tech','troubleshooting','engineer','lead','engineer','specialist','system']
allinlist = ['customer','support|care']
notinlist = ['production','engineer','analyst','voice','chat','devops']# founder

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
                df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]
apl = set(filtered_df['job_title'].str.lower())
titledict['Customer Support'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['fluent','language','voice','call','calling','tele','chat','voice','chat','process']
allinlist = ['voice|process','voice']
notinlist = ['support']

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
                df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]
apl = set(filtered_df['job_title'].str.lower())
titledict['Voice Process'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['soft','java','full','stack','back','end','developer','front','engineer','fullstack.net','dot','sap','sql']
allinlist = ['developer|engi|java|stack|soft|front|back|full|fullstack.net|dot|sap|sql','developer|engineer']
notinlist = ['data','anal']# founder

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
                df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

apl = set(filtered_df['job_title'].str.lower())
titledict['Software Development'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['medical','pharmacist','coding','coder']
# allinlist = ['cod|medical|pharmacist|coding|coder']
# notinlist = ['equity']

# allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

# filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
#                 df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
#                 ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

# apl = set(filtered_df['job_title'].str.lower())
# titledict['medical coder'] = apl
# #------------------------------------------------------------------------------------------------------
# anyinlist = ['medical']
# allinlist = ['bill','med']
# notinlist = []# founder
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['medical billing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sales','inbound','tele','inside','customer','relationship','engineer','perfomance','analyst','it','tech',
             'executive','marketing']
allinlist = ['sales|executive|marketing|manager|territory']
notinlist = ['recruiter','support','care','development']

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
                df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

apl = set(filtered_df['job_title'].str.lower())
titledict['Sales'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['sales','inbound','tele','inside','customer','relationship','engineer','perfomance','analyst','it','tech',
             'executive','marketing']
allinlist = ['business|development','development']
notinlist = ['recruiter','support','care','sales']

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
                df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

apl = set(filtered_df['job_title'].str.lower())
titledict['Business Development'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','anal','machine','learning','scie','data','anal','ai/ml']
allinlist = ['data|anal||scien|ai|ml']
notinlist = ['admin','video','engineer','engi']

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'(?:' + '|'.join(anyinlist) + r')', regex=True) &
                df['job_title'].str.lower().str.contains(r'(?:' + '|'.join(allinlist) + r')', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'(?:' + '|'.join(notinlist) + r')', regex=True)]

apl = set(filtered_df['job_title'].str.lower())
titledict['Data Scientist'] = apl

#------------------------------------------------------------------------------------------------------
anyinlist = ['data','anal','ml','machine','learning','engi','data','ai/ml']
allinlist = ['data|engi|ai|ml|engineer']
notinlist = ['admin','video','scien']

allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

filtered_df = df[df['job_title'].str.lower().str.contains(r'(?:' + '|'.join(anyinlist) + r')', regex=True) &
                df['job_title'].str.lower().str.contains(r'(?:' + '|'.join(allinlist) + r')', regex=True) &
                ~df['job_title'].str.lower().str.contains(r'(?:' + '|'.join(notinlist) + r')', regex=True)]

apl = set(filtered_df['job_title'].str.lower())
titledict['Data Engineer'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['devops']
allinlist = ['devops']
notinlist = []
# notinlist.extend(list(titledict['it recruiter']))
apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Devops'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['project','manager','soft','tele','it']
allinlist = ['project','mana',]
notinlist = ['- projects','manager project','non-it','non it']
# notinlist.extend(list(titledict['it recruiter']))
apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Project Management'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['digital','marketing','campaigns','ads','media']
allinlist = []
notinlist = []
# notinlist.extend(list(titledict['it recruiter']))
apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Digital Marketing'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['counsellor','counselor'] 
allinlist = []
notinlist = []
# notinlist.extend(list(titledict['it recruiter']))
apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Counsellor'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['content','writer','copy']
# allinlist = ['writer']
# notinlist = []
# # notinlist.extend(list(titledict['it recruiter']))
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['content/copy writer'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['graphic','designer']
# allinlist = ['designer']
# notinlist = []
# # notinlist.extend(list(titledict['it recruiter']))
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['graphic designer'] =  apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['architect']
# allinlist = ['architect']
# notinlist = []
# # notinlist.extend(list(titledict['it recruiter']))
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['architect'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['business','analyst']
allinlist = ['business','analyst']
notinlist = []
# notinlist.extend(list(titledict['it recruiter']))

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)

titledict['Business Analyst'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['ui','ux','designer','developer']
# allinlist = ['ui|ux']
# notinlist = ['java']

# allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

# filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
#                 df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
#                 ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

# apl = set(filtered_df['job_title'].str.lower())
# titledict['ui/ux designer'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['accounts']
# allinlist = ['accounts','mana']
# notinlist = ['store','ass']
# # notinlist.extend(list(titledict['it recruiter']))

# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['accounts manager'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['operations']
# allinlist = ['operations','lead|executive']
# notinlist = ['stack','front','back','soft']# founder

# allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

# filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
#                 df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
#                 ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

# apl = set(filtered_df['job_title'].str.lower())
# titledict['operations'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['engineer']
allinlist = ['engineer']
notinlist = []
# notinlist.extend([item for seet in titledict for item in titledict[seet]])

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Engineer'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['hr','recruiter','recru','relationship','human',]
allinlist = []
notinlist = []
# notinlist.extend(list(titledict['it recruiter']))
apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Recruiter'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['product','management','manager']
allinlist = ['product','manager']
notinlist = ['admission','hemis','missi','soft','java','relationship','production']

# notinlist.extend(list(titledict['it recruiter']))
apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
      apl.add(i)
titledict['Product Management'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['equity','dealer','finance','advisor','financial']
# allinlist = ['equity|trader|advisor|financial']
# notinlist = ['admission','hemis','missi','soft','java','relationship','production']

# allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

# filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
#                 df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
#                 ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

# apl = set(filtered_df['job_title'].str.lower())
# titledict['equity advisor'] = apl
#------------------------------------------------------------------------------------------------------
# anyinlist = ['accountant','account']
# allinlist = []
# notinlist = []

# # notinlist.extend(list(titledict['it recruiter']))
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# titledict['accountant'] = apl

In [51]:

anyinlist = ['mis']
allinlist = []
notinlist = ['admission','hemis','missi','soft','customer','voice','it','mist','lyst','ern','port','list','ciate']

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl 
titledict['Lead/Management'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['manager']
allinlist = []
notinlist = ['admission','hemis','missi','soft','customer','voice','it','mist','lyst','ern','port','list','ciate']

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
titledict['Lead/Management'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['lead']
allinlist = []
notinlist = ['admission','hemis','missi','soft','customer','voice','it','mist','lyst','ern','port','list','ciate','ing','tech','net','java']

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
titledict['Lead/Management'].update(apl)
#------------------------------------------------------------------------------------------------------
# ,'manage','lead','executive','head'
anyinlist = ['executive','head']
allinlist = []
notinlist = ['admission','hemis','missi','soft','customer','voice','it','mist','lyst','ern','port','list','ciate','ing','tech','net','java']

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
titledict['Lead/Management'].update(apl)
#------------------------------------------------------------------------------------------------------
# ,'manage','lead','executive','head'
anyinlist = ['support','care','cust','tech','troubl','engineer','specialist','system']
allinlist = ['custo']
notinlist = ['admission','hemis','missi','soft','it','mist','lyst','ern','port','list','ciate','ing','tech','net','java']

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
titledict['Customer Support'].update(apl)
#------------------------------------------------------------------------------------------------------
# ,'manage','lead','executive','head'
anyinlist = ['support','care','tech','troubl','engineer','specialist','system']
allinlist = ['voice']
notinlist = ['cust','missi','soft','it',]

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
titledict['Voice Process'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['busine','dev']
allinlist = ['busine','dev']
notinlist = []

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)
apl
titledict['Business Development'].update(apl)
#------------------------------------------------------------------------------------------------------


In [52]:
anyinlist = ['busine','dev']
allinlist = ['busine','dev']
notinlist = []

apl = set()
for i in df['job_title']:
    i = i.lower()
    if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
        apl.add(i)


In [53]:
len(titledict.keys())

16

In [54]:
for i in titledict.keys():
    print(i)

Lead/Management
Customer Support
Voice Process
Software Development
Sales
Business Development
Data Scientist
Data Engineer
Devops
Project Management
Digital Marketing
Counsellor
Business Analyst
Engineer
Recruiter
Product Management


In [55]:
df['job_title']

0                                      Aws Devops Engineer
1        Hiring B. Tech Freshers || Hyderabad || Excell...
2        Front End Engineer/ Front End Engineer Lead - ...
3                 Day shift - Hindi Voice Process - 3L CTC
4        Great Salaries For Tech Support in Bangalore -...
                               ...                        
52730                                 Salesforce Developer
52731                                      Ui Ux Developer
52732                      Software Engineer And Developer
52733                      Business Intelligence Developer
52734                                    Marketing Analyst
Name: job_title, Length: 52735, dtype: object

In [56]:
anyinlist = ['accountant']
allinlist = []
notinlist = []

# notinlist.extend(list(titledict['it recruiter']))
# apl = set()
# for i in df['job_title']:
#     i = i.lower()
#     if all(word in i for word in allinlist) and all(word not in i for word in notinlist) and any(word in i for word in anyinlist):
#       apl.add(i)
# apl
# allinlist = [r'(?:' + '|'.join(element.split('|')) + ')' for element in allinlist]

# filtered_df = df[df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(anyinlist) + r')\b', regex=True) &
#                 df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(allinlist) + r')\b', regex=True) &
#                 ~df['job_title'].str.lower().str.contains(r'\b(?:' + '|'.join(notinlist) + r')\b', regex=True)]

# apl = set(filtered_df['job_title'].str.lower())
# for i in apl:
#     print(i)

In [57]:
converted = pd.DataFrame.from_dict(titledict, orient='index').T
converted.to_excel('title_maps.xlsx',index = False)

In [58]:
reverse_mapping = {title.lower(): category for category, titles in titledict.items() for title in titles}
def map_title(value):
    lower_value = value.lower()
    return reverse_mapping.get(lower_value, 'unclassified')

df['mapped_job_title'] = df['job_title'].apply(map_title)

In [59]:
f = []
for k,v in df[df['mapped_job_title']=='unclassified']['job_title'].value_counts().items():
    # f.append(list(k,v))
    l = []
    l.append(k)
    l.append(v)
    f.append(l)
fdf = pd.DataFrame(f,columns = ['role','count'])

In [60]:
fdf 

,role,count
0,Graphic Designer,227
1,Interior Designer,55
2,Talent Acquisition Specialist,53
3,Tele Caller,46
4,Equity Dealer,40
...,...,...
7484,NJS2897 - Associate Consultant Neurologist (DM...,1
7485,Freshers,1
7486,Experienced International BPO Agent-Urgently r...,1
7487,Technical Maintenance Leader,1


In [61]:
fdf.to_excel('unmapedroles.xlsx',index = False)

In [62]:
value_counts = {}
counts = df['mapped_job_title'].value_counts()
for value, count in counts.items():
    if value in value_counts:
        value_counts[value] += count
    else:
        value_counts[value] = count
value_counts

{'unclassified': 10745,
 'Sales': 9488,
 'Software Development': 6198,
 'Recruiter': 5078,
 'Engineer': 5048,
 'Lead/Management': 4619,
 'Digital Marketing': 3521,
 'Business Development': 2420,
 'Customer Support': 1385,
 'Voice Process': 1228,
 'Data Scientist': 1048,
 'Data Engineer': 820,
 'Counsellor': 492,
 'Business Analyst': 282,
 'Project Management': 171,
 'Product Management': 111,
 'Devops': 81}

In [63]:
# converted = pd.DataFrame.from_dict(value_counts, orient='index')
converted = pd.DataFrame.from_dict(value_counts, orient='index', columns=['Count'])
converted.index.name = "Roles"
converted.to_excel('title_value_counts.xlsx')
converted

,Count
Roles,
unclassified,10745
Sales,9488
Software Development,6198
Recruiter,5078
Engineer,5048
Lead/Management,4619
Digital Marketing,3521
Business Development,2420
Customer Support,1385


In [64]:
df[['job_title','mapped_job_title']].to_excel('titles and maps.xlsx',index = False)

In [65]:
df.columns

Index(['job_title', 'company_name', 'experience', 'salary', 'location',
       'description', 'posted_on', 'current_date', 'skill1', 'skill2',
       'skill3', 'skill4', 'skill5', 'skill6', 'skill7', 'skill8', 'role',
       'industry type', 'department', 'employment type', 'role category',
       'job_description', 'mapped_average_sal', 'avg_experience',
       'mapped_job_title', 'mapped_job_title_1'],
      dtype='object')

In [66]:
skilcol = ([col for col in df.columns if 'skil' in col])
df['combined_skills'] = df[skilcol].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)
df['combined_skills'] = df['combined_skills'].str.lower()
for col in df[skilcol].columns:
    df[col] = df[col].str.lower()

In [67]:
# for i in df['combined_skills']:
#     print(i)
df['combined_skills'] = df['combined_skills'].str.replace(r'^[A-Za-z0-9,.+]+$',' ',regex=True).str.lower().str.replace(r'very|good','',regex=True)
# uniqueskill = set()
# for i in df['combined_skills']:
#     tlis = i.split(',')
#     for j in tlis:
#         uniqueskill.add(j)

In [68]:
# skills = list(uniqueskill)

# json_string = json.dumps(skills, ensure_ascii=False)

# with open('uniqueskills.json', 'w', encoding='utf-8') as json_file:
#     json_file.write(json_string)

# checkpoint

In [69]:
# safeddf = df[skilcol]

In [70]:
# df[skilcol] = safeddf

In [71]:
# # removing skills that are not in the list.

# t = set()
# for i in df[skilcol]:
#     for j in df[i]:
#         try:
#             if 'skill' in j and all(word not in j for word in['python', 'it','azure','cloud','microsoft office','technical','budgeting','audit','reporting','visualization','clinical','programming','automation']):
#                 t.add(j)
#         except:
#             None
            

In [72]:
# def skilrem(value):
#     if value in t:
#         return None
#     return value

# for col in df[skilcol].columns:
#     df[col] = df[col].apply(skilrem)

In [73]:
# faulwd = set()
# t = pd.Series(list(t))
# for  i in t.str.replace(r'skills|skilled|skill|', '',regex=True).str.replace(r'[^A-Za-z0-9]', ' ',regex=True).str.strip():
#     faulwd.add(i)

# mistset = set()
# for i in faulwd:
#     i = i.split(' ')
#     for j in i:
#         mistset.add(j)

# # for i in mistset:
# #     print(i)
# mistset.discard('office')
# mistset.discard('selling')
# mistset.discard('dba')
# mistset.discard('marketing')
# mistset.discard('seo')

In [74]:
# def skilrem(value):
#     if value != None:    
#         if isinstance(value,float):
#             return None        
#         if len(value) < 2 and (value != 'c'):
#             return None 
#         if value in mistset:
#             return None
#     return value 
        
# for col in df[skilcol].columns:
#     df[col] = df[col].apply(skilrem)

In [75]:
#removal cases. rename the key to ''.

In [76]:
# errordict['hybris'] = errordict.pop('# hybris')
# errordict['sketchup'] = errordict.pop('sketchup.')
# errordict['project management'] = errordict.pop('. project management')

# payscale wise

In [77]:
skills = set()
for ind,val in df[skilcol].iterrows():
    val = val.to_list()
    skills.update(set(val))
len(skills)

31713

In [78]:
'''Lead/Management
Customer Support
Voice Process
Software Development
Sales
Business Development
Data Scientist
Data Engineer
Devops
Project Management
Digital Marketing
Counsellor
Business Analyst
Engineer
Recruiter
Product Management'''

'Lead/Management\nCustomer Support\nVoice Process\nSoftware Development\nSales\nBusiness Development\nData Scientist\nData Engineer\nDevops\nProject Management\nDigital Marketing\nCounsellor\nBusiness Analyst\nEngineer\nRecruiter\nProduct Management'

In [79]:
df.to_csv('checkpoint.csv',index=False)

In [80]:
df = pd.read_csv('checkpoint.csv')#------------------------------------------------------------------------------------------------------

In [81]:
skills_edited = {}

In [82]:
anyinlist = ['statistical','analysis','stat','analyst']
allinlist = ['statis']
notinlist = ['license','meeting','purlin','designs','maint','sales','concrete','drilling','cnc','bio']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Statistical Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','modeling']
allinlist = ['model']
notinlist = ['3d','char','2d','equipme','surf','solid']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Modeling'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['big','data']
allinlist = ['big','data']
notinlist = ['3d','char','2d','equipme','surf','solid']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Big Data Analytics'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['deep','learning']
allinlist = ['deep','learning']
notinlist = ['3d','char','2d','equipme','surf','solid']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Deep Learning'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['nlp','natural','language','processing']
allinlist = ['nlp']
notinlist = ['3d','char','2d','equipme','surf','solid']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Natural Language Processing (NLP)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ai']
allinlist = ['ai','artificial']
notinlist = ['3d','char','2d','equipme','surf','solid']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Artificial Intelligence (AI)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','visualization']
allinlist = ['visu']
notinlist = ['3d','char','2d','equipme','surf','solid','archit','creat','exteri']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Visualization'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['tableau']
allinlist = ['tableau']
notinlist = ['3d','char','2d','equipme','surf','solid','archit','creat','exteri']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Tableau'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sas']
allinlist = []
notinlist = ['3d','char','2d','equipme','surf','solid','archit','creat','exteri','disassembling','sass','sase','esas','sast','hsas','ssas']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAS'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['algorithm','development']
allinlist = ['algorithm','development']
notinlist = ['3d','char','2d','equipme']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Algorithm Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['computer','vision']
allinlist = ['computer','vision']
notinlist = ['3d','char','2d','equipme','surf','solid','archit','creat','exteri','disassembling','sass','sase','esas','sast','hsas','ssas']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Computer Vision'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['spark']
allinlist = ['spark']
notinlist = ['3d','char']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Spark'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','min']
allinlist = ['data','min']
notinlist = ['3d','char','ministra','ming','dmin']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Mining'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['etl']
allinlist = ['etl']
notinlist = ['3d','char','ministra','ming','dmin','targetl']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['ETL (Extract, transform, load)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['azure']
allinlist = ['azure']
notinlist = ['3d','char','ministra','ming','dmin','targetl']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Azure'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','warehou','dwh']
allinlist = ['dwh']
notinlist = ['3d','char','ministra','ming','dmin','targetl']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Warehouse'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','warehou']
allinlist = ['data','warehou']
notinlist = ['3d','char','ministra','ming','dmin','targetl']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Warehouse'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['google','cloud']
allinlist = ['google','cloud',]
notinlist = ['3d','char','ministra','ming','dmin','targetl']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Google Cloud Platform (GCP)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['gcp']
allinlist = ['gcp',]
notinlist = ['3d','char','ministra','ming','dmin','targetl','mgcp','ich']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Google Cloud Platform (GCP)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['power','bi']
allinlist = ['power','bi']
notinlist = ['3d','char','ministra','ming','dmin','targetl','mgcp','ich']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Power BI'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['scala']
allinlist = ['scala']
notinlist = ['3d','char','ministra','escala','abili','scalar','calable']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Scala'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['snowflake']
allinlist = ['snowflake']
notinlist = ['3d','char','ministra','escala','abili','scalar','calable']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Snowflake Cloud Data Platform'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['hadoop']
allinlist = ['hadoop']
notinlist = ['3d','char','ministra','escala','abili','scalar','calable']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Apache Hadoop'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['devops']
allinlist = ['devops']
notinlist = ['3d','char','ministra','escala','abili','scalar','calable']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Development Operations (DevOps)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['automation','scripting']
allinlist = ['automation']
notinlist = ['appian', 'anywhere', 'sales', 'studio', 'azure', 'bdd', 'test', 'boardroom', 'build', 'cypress', 'engineer', 'home', 'hubspot', 'marketing', 'hvac', 'industrial', 'kotlin', 'backend', 'machine', 'marketing', 'martech', 'mis', 'office', 'perl', 'plc', 'quip', 'sap', 'security', 'smoke', 'soar', 'substation', 'ui', 'vba', 'web', 'wiser', 'workflow']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Automation Scripting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['jenkins']
allinlist = ['jenkins']
notinlist = ['sales']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Jenkins'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['terraform']
allinlist = ['terraform']
notinlist = ['sales']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Terraform'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['docker']
allinlist = ['docker']
notinlist = ['sales']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Docker'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['git']
allinlist = ['git']
notinlist = ['sales','digi','longi','logi','gito']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['GIT'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cloud','compu']
allinlist = ['cloud','compu']
notinlist = ['sales','digi','longi','logi','gito']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Cloud Computing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['power','shell']
allinlist = ['power','shell']
notinlist = ['sales','digi','longi','logi','gito']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Windows PowerShell'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sys','admin']
allinlist = ['sys','admin']
notinlist = ['sales','digi','longi','logi','gito']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['System Administration'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['red','hat']
allinlist = ['red','hat']
notinlist = ['sales','digi','longi','logi','gito','chatered']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Red Hat Ansible'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['recruit']
allinlist = ['recruit']
notinlist = ['sales','digi','longi','logi','gito','chatered']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Recruiting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ats']
allinlist = ['ats']
notinlist = ['cat','fat','hat','mat','lat']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Applicant Tracking System (ATS) Software'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['appli']
allinlist = ['appli','track']
notinlist = ['cat','fat','hat','mat','lat']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Applicant Tracking System (ATS) Software'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['sourcing']
allinlist = ['sourcing']
notinlist = ['garment','import','estate','resourc']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Sourcing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['communi']
allinlist = ['communi']
notinlist = ['garment','import','estate','resourc','analog','unity']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Oral / Verbal Communication'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['hir']
allinlist = ['hir']
notinlist = ['garment','import','estate','resourc','analog','unity','chira','ethir','fhir','third']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Hiring'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['hr']
allinlist = ['hr']
notinlist = ['garment','import','estate','resourc','analog','unity','chira','ethir','fhir','third','ehr','phr','bphr','bhr','chro','hram','thr','ahr','shr','whrb','zhr']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Human Resources (HR)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['hum','res']
allinlist = ['hum','res']
notinlist = ['garment','ehr','phr','bphr','bhr','chro','hram','thr','ahr','shr','whrb','zhr']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Human Resources (HR)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['cust','ser']
allinlist = ['cust','ser']
notinlist = ['garment','ehr','phr','bphr','bhr','chro','hram','thr','ahr','shr','whrb','zhr']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Customer Service'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['client']
allinlist = ['client']
notinlist = ['garment','ehr','phr','bphr','bhr','chro','hram','thr','ahr','shr','whrb','zhr','scripts','server']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Client Interaction'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sales']
allinlist = ['sales']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['peop','manag']
allinlist = ['peop','manag']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['People Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['crm']
allinlist = ['crm']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Customer Relationship Management (CRM)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['custome','relat','manag']
allinlist = ['custome','relat','manag']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Customer Relationship Management (CRM)'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['training']
allinlist = ['training']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Training'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['negotia']
allinlist = ['negotia']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Negotiation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['soci','media','marke']
allinlist = ['soci','media','marke']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Social Media Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['emplo','rela']
allinlist = ['emplo','rela']
notinlist = ['sour','anal','couns','archi','hir']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Employee Relations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['m','offi']
allinlist = ['m','offi']
notinlist = ['sour','anal','couns','archi','hir','mid','med','manag','enga','migr','chief','front','auto','admin','ine','mily','quip','rmal','plie','ruitme',
             'tim','room']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Office'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['admin']
allinlist = ['admin']
notinlist = ['sour']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Administration'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['schedu']
allinlist = ['schedu']
notinlist = ['inves','machi','produ','oject','bar']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Scheduling'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['busin','dev']
allinlist = ['busin','dev']
notinlist = ['inves','machi','produ','oject','bar']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Business Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['bd']
allinlist = ['bd']
notinlist = ['inves','machi','produ','oject','bar','bdd','bda','bbd','bdc','bdg','it','mbd','fbd','nbd','obd','ebd','info']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Business Development'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['prob','sol']
allinlist = ['prob','sol']
notinlist = ['inves']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Problem Solving'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['troublesh']
allinlist = ['troublesh']
notinlist = ['inves']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Troubleshooting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['call','cent']
allinlist = ['call','cent']
notinlist = ['inves']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Tech Support Call Center'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['desk','help']
allinlist = ['desk','help']
notinlist = ['inves']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Help Desk / Desktop Support'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data','entr']
allinlist = ['data','entr']
notinlist = ['inves','ope']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Entry'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['excel']
allinlist = ['excel']
notinlist = ['inves','ope','lence','lant','lent']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Excel'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['opera','mana']
allinlist = ['opera','mana']
notinlist = ['inves','lence','lant','lent']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Operations Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['it','suppo']
allinlist = ['it','suppo']
notinlist = ['inves','lence','lant','lent','dig','equi','onsi','recr']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Information Technology (IT) Support'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['tech','suppo']
allinlist = ['tech','suppo']
notinlist = ['inves','lence','lant','lent','dig','equi','onsi','recr']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Information Technology (IT) Support'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['tech','wri']
allinlist = ['tech','wri']
notinlist = ['inves',]

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Technical Writing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['bus','anal']
allinlist = ['bus','anal']
notinlist = ['inves',]

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Business Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['req','ana']
allinlist = ['req','ana']
notinlist = ['inves','mana']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Requirements Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['proj','mana']
allinlist = ['proj','mana']
notinlist = ['inves',]

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Project Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['proc','impr']
allinlist = ['proc','impr']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Process Improvement'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['agile']
allinlist = ['agile']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Agile Software Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sale','forc']
allinlist = ['sale','forc']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Salesforce'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['prod','manag']
allinlist = ['prod','manag']
notinlist = ['ion']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Product Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['documentation']
allinlist = ['documentation']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Documentation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['visio']
allinlist = ['visio']
notinlist = ['ion','or']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Visio'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['digi','mark']
allinlist = ['digi','mark']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Digital Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['seo']
allinlist = ['seo']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Search Engine Optimization (SEO)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sear','engi','opt']
allinlist = ['sear','engi','opt']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Search Engine Optimization (SEO)'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cust','sale','camp','mark']
allinlist = ['emai']
notinlist = ['comm','chat','mana']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Email Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['goog']
allinlist = ['anal']
notinlist = ['comm','chat','mana']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Google Analytics'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['web','cont']
allinlist = ['web','cont']
notinlist = ['dev']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Web Content Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sem']
allinlist = ['sem']
notinlist = ['emi','semb','seme','rus','tse']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Search Engine Marketing (SEM)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sear','engi','mark']
allinlist = ['sear','engi','mark']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Search Engine Marketing (SEM)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['ad','word']
allinlist = ['ad','word']
notinlist = ['pad']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Google Ad Words'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['paid','medi']
allinlist = ['paid','medi']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Paid Media Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['stra','mark']
allinlist = ['stra','mark']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Strategic Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['grap','desi']
allinlist = ['grap','desi']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Graphic Design'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cont','mana']
allinlist = ['cont','mana']
notinlist = ['web','act','rol']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Content Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mark','comm']
allinlist = ['mark','comm']
notinlist = ['odi','ercial']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Marketing Communications'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['web','desi']
allinlist = ['web','desi']
notinlist = ['dev']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Web Design'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['vid','edit']
allinlist = ['vid','edit']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Video Editing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['copy','writ']
allinlist = ['copy','writ']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Copywriting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['onlin','mark']
allinlist = ['onlin','mark']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Online Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['pay','click']
allinlist = ['pay','click']
notinlist = []

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Pay-Per-Click Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['web','anal']
allinlist = ['web','anal']
notinlist = ['dev']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Web Analytics'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['java']
allinlist = []
notinlist = ['node','react','script','js']

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Java'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['java','js']
allinlist = []
notinlist = ['node','react','jso','angu','fast']
notinlist.extend( list(skills_edited['Java']))
notinlist.remove('java')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Java Script'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sql']
allinlist = []
notinlist = ['my','no','pl','post','pg','ps']
# notinlist.extend( list(skills_edited['Java']))
# notinlist.remove('java')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SQL'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['python']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['Java']))
# notinlist.remove('java')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Python'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['c']
allinlist = ['c']
notinlist = ['node','react','#','hash','&','/','sharp','license','meeting','purlin',
             'uce','uct','clo','co','cr','cal','nce','ce','ck','cu','cm','ch','ct','ci','ca','cl','ic','cn','cp','bc','2c','oc','ac','sc','gc','yc','vc',
             'rc','lc','ec','tc','pc','cf','cg','cs','cq','cg','cy','dc','mc','9c','qc','cd','cc','nc','wc','c1','c2','ch','cv','c4','cb','c3','cw','cx',
             'uc','fc','hc','4c','+','plus','&']
# notinlist.extend( list(skills_edited['Java']))
# notinlist.remove('java')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['C Programming Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['c']
allinlist = ['c']
notinlist = ['node','react','#','hash','&','/','sharp','license','meeting','purlin',
             'uce','uct','clo','co','cr','cal','nce','ce','ck','cu','cm','ch','ct','ci','ca','cl','ic','cn','cp','bc','2c','oc','ac','sc','gc','yc','vc',
             'rc','lc','ec','tc','pc','cf','cg','cs','cq','cg','cy','dc','mc','9c','qc','cd','cc','nc','wc','c1','c2','ch','cv','c4','cb','c3','cw','cx',
             'uc','fc','hc','4c','&']
notinlist.extend( list(skills_edited['C Programming Language']))
notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['C++ Programming Language'] = apl 
skills_edited['C++ Programming Language'].remove('c')
#------------------------------------------------------------------------------------------------------
anyinlist = ['c']
allinlist = ['c']
notinlist = ['license','meeting','purlin',
             'uce','uct','clo','co','cr','cal','nce','ce','ck','cu','cm','ch','ct','ci','ca','cl','ic','cn','cp','bc','2c','oc','ac','sc','gc','yc','vc',
             'rc','lc','ec','tc','pc','cf','cg','cs','cq','cg','cy','dc','mc','9c','qc','cd','cc','nc','wc','c1','c2','ch','cv','c4','cb','c3','cw','cx',
             'uc','fc','hc','4c','&']
notinlist.extend( list(skills_edited['C Programming Language']))
notinlist.extend( list(skills_edited['C++ Programming Language']))
notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['C# Programming Language'] = apl 
skills_edited['C# Programming Language'].remove('c')
#------------------------------------------------------------------------------------------------------
anyinlist = ['react']
allinlist = ['react']
notinlist = ['or','on']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['React.js'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['node']
allinlist = []
notinlist = ['fet','desi','lymph','eb']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Node.js'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['angul']
allinlist = []
notinlist = ['js']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Angular'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['aws']
allinlist = []
notinlist = ['law','jaw','dev']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Amazon Web Services (AWS)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['amaz','web']
allinlist = ['amaz','web']
notinlist = ['law','jaw','dev']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Amazon Web Services (AWS)'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['net','dot']
allinlist = []
notinlist = ['law','jaw','etwor','ado','bnet','trix','iler','ernet','etba','cnet','etsc','netru','inet','enet','menet','gnet','enet','vnet',
             'ranet','onet','snet','netcr','pnet','rdot','nets','netx','netw','neti','netc','netap','gh net','asp']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['.NET'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['linux']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Linux'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['spring']
allinlist = []
notinlist = ['boot']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Spring'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['spring']
allinlist = ['boot']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Spring Boot'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['gola']
allinlist = ['go']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Go (Golang) Programming Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['kubern']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Kubernetes'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['php']
allinlist = []
notinlist = ['ephp','px','pc','ophp',]
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['PHP'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ruby','ror','rails']
allinlist = []
notinlist = ['rror','cror','drail','uror','oror','grail',]
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Ruby on Rails'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['html']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['HTML'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['machine learning','machinelearning','ml']
allinlist = []
notinlist = ['tml','aml','cont','machines','know','ancing','bsc','mlt','bml','sale','cnc','oper','messure','concre','xml','dml','hydro','drill','e-','elearning',
             'bend','measu','motiv','lamina','elec','design','garme','grin','heav','hmc','indu','inje','joom','cuttin','lathe','rawi','build','automa','assem',
             'inff','inte','mlc','mlf','mlo','qml','gml','sml','uml']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Machine Learning'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['data']
allinlist = ['anal']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Data Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['nego']
allinlist = ['contr']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Contract Negotiation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['comp']
allinlist = ['legal']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Legal Compliance'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['corp']
allinlist = ['gov']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Corporate Governance'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['comp']
allinlist = ['regu']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Regulatory Compliance'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['litiga','case']
allinlist = ['manag']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Litigation Case Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['contra']
allinlist = ['manag']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Contract Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['merge','acquit']
allinlist = []
notinlist = ['eme','code','mail']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Mergers and Acquisitions'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mana','cont']
allinlist = ['risk']
notinlist = ['eme','code','mail']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Risk Management / Risk Control'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['doc','revi']
allinlist = ['legal']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Legal Document Review'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['intelle']
allinlist = []
notinlist = ['gent']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Intellectual Property (IP)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['leade']
allinlist = ['ship']
notinlist = ['support']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Leadership'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['commercial']
allinlist = []
notinlist = ['support']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Commercial'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['real']
allinlist = ['estate']
notinlist = ['support']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Real Estate'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['litig']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Complex Litigation Case Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['lead']
allinlist = ['team','ship']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Team Leadership'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['peop']
allinlist = ['manag']
notinlist = ['connect to people']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['People Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['labor']
allinlist = ['rela']
notinlist = ['connect to people']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Labor Relations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['dev']
allinlist = ['prod']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Product Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['strate']
allinlist = ['plan']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Strategic Planning'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['strate']
allinlist = ['busi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.extend( list(skills_edited['C++ Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Business Strategy'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['prod']
allinlist = ['mark']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Product Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['rese']
allinlist = ['mark']
notinlist = ['relat']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Market Research'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mana','rela']
allinlist = ['vend']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Vendor Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['des']
allinlist = ['ux']
notinlist = ['ui','lin']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['User Experience (UX) Design'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['anal']
allinlist = ['mark']
notinlist = ['bench','stock','equi','fin']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Market Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['engineerin']
allinlist = ['desi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Engineering Design'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['proj']
allinlist = ['co','ordi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Project Coordinating'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['client']
allinlist = ['intera']
notinlist = ['view']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Client Interaction'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['budg',]
allinlist = ['manag']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Budget Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['proc']
allinlist = ['improv']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Process Improvement'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['const']
allinlist = ['est']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Construction Estimating'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['assu','cont']
allinlist = ['qual']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Quality Assurance / Quality Control'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ms','micro']
allinlist = ['proj']
notinlist = ['tems']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Project'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['des','cad']
allinlist = ['aut']
notinlist = ['ted','tion','tive']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Autodesk AutoCAD'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['change']
allinlist = ['manag']
notinlist = ['ex']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Change Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['consul']
allinlist = ['manag']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Consulting Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['prog']
allinlist = ['manag']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Program Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['organi']
allinlist = ['dev']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Organizational Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['fina']
allinlist = ['anal']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Financial Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['serv']
allinlist = ['cust']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Customer Service'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cent']
allinlist = ['call']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Call Center Operations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['inbou']
allinlist = ['call']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Inbound Calls'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['relatio']
allinlist = ['cust']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Customer Relations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['typing']
allinlist = []
notinlist = ['prot']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Typing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['outbo']
allinlist = ['call']
notinlist = ['prot']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Outbound Calls'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['eng']
allinlist = ['lang']
notinlist = ['engi']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['English Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['insuran']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Insurance'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['medi']
allinlist = ['termi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Medical Terminology'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['banking']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Banking'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['javascript']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Java Script'].update(apl) 
#------------------------------------------------------------------------------------------------------
anyinlist = ['inter']
allinlist = ['clien']
notinlist = ['tional']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Client Interaction'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['gen']
allinlist = ['lead']
notinlist = ['gency','ship','urgen']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Lead Generation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['b2b']
allinlist = ['sales']
notinlist = ['gency','ator','auto','hire','local','target','customer','cutting','tool','desktop','door','tolo','ptop','oto','sto']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Business to Business (B2B) Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mana']
allinlist = ['acco']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Account Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['stra']
allinlist = ['part']
notinlist = ['admin']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Strategic Partners'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['strat']
allinlist = ['sale']
notinlist = ['admin','meet']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Strategic Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['insi']
allinlist = ['sale']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Inside Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['tech']
allinlist = ['sale']
notinlist = ['logy','ques','ed','solut','data','math']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Technical Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mana']
allinlist = ['sale']
notinlist = ['logy','ques','ed','solut','data','math']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Sales Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['dev']
allinlist = ['prod']
notinlist = ['logy','ques','ed','solut','data','math']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Product Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['dev']
allinlist = ['web']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Web Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['js']
allinlist = ['vue']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Vue.js'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sql']
allinlist = ['my']
notinlist = ['amy','emy']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['MySQL'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['scr']
allinlist = ['typ']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Typescript'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['go']
allinlist = ['djan']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Django'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['js']
allinlist = ['angular']
notinlist = ['reac','vue']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Angular.js'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['asp']
allinlist = ['net']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['ASP.NET Framework'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cycl']
allinlist = ['ful','recru']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Full-Cycle Recruiting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['post']
allinlist = ['sql']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['PostgreSQL'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['db','go']
allinlist = ['mong']
notinlist = ['chef','oose']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['MongoDB'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['micro']
allinlist = ['serv']
notinlist = ['microsoft','iser']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microservices'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['nod','js']
allinlist = ['exp']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Express (Node.js)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['field']
allinlist = ['sales']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Field Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['presen']
allinlist = []
notinlist = ['presence']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Presentations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sup']
allinlist = ['desk',]
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Help Desk / Desktop Support'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['manag','admi']
allinlist = ['netw']
notinlist = ['menet','relation']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Network Management / Administration'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['activ']
allinlist = ['dire']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Active Directory'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['netw']
allinlist = ['sup']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Network Support'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['inst','upgr']
allinlist = ['soft']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Software Installation & Upgrade'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sup']
allinlist = ['wind']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Windows Support'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['soft']
allinlist = ['know']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Software Knowledge'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['exchan']
allinlist = []
notinlist = ['case','email','admin','heat','online','eign','link','stoc','data']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Microsoft Exchange'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['serv']
allinlist = ['wind']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Windows Server'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sale']
allinlist = ['chan']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Channel Sales'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sale']
allinlist = ['admin']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Sales Administration'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['finan']
allinlist = ['anal']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Financial Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['finan']
allinlist = ['mode']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Financial Modeling'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['invest']
allinlist = ['mana']
notinlist = ['reg']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Investment Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['reconciliation']
allinlist = ['reconciliation']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Reconciliation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['repo']
allinlist = ['fina']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Financial Reporting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['investor']
allinlist = ['relat']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Investor Relations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['busine']
allinlist = ['intel']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Business Intelligence'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sql']
allinlist = ['pl']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['PL/SQL'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['scrip']
allinlist = ['shell']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Shell Scripting'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['team']
allinlist = ['mana']
notinlist = ['time']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Team Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['merch']
allinlist = []
notinlist = ['man']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Merchandising'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['com']
allinlist = ['loan']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Commercial Loans'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['fin']
allinlist = ['adv']
notinlist = ['finfet']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Financial Advisor'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['retail']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Retail'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cash']
allinlist = ['han']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Cash Handling'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mana']
allinlist = ['inven']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Inventory Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['organiz']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Organizing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['fork']
allinlist = ['li']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Forklift Certified'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['span']
allinlist = ['lang']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Spanish Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['maint']
allinlist = ['nance']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Maintenance'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['design']
allinlist = ['reta']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Design'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cyber']
allinlist = ['sec']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Cyber Security'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['risk']
allinlist = ['sec']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Security Risk Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['inf']
allinlist = ['sec','stru']
notinlist = ['tech']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['IT Security & Infrastructure'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['test','audi']
allinlist = ['sec']
notinlist = ['tech']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Security Testing and Auditing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['net']
allinlist = ['sec']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Network Security Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['27001']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['ISO 27001'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['info','even','siem']
allinlist = ['secu']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Security Information and Event Management (SIEM)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['siem']
allinlist = ['siem']
notinlist = ['iemens']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Security Information and Event Management (SIEM)'].update(apl)
#------------------------------------------------------------------------------------------------------
anyinlist = ['vulne']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Vulnerability Assessment'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['penet']
allinlist = []
notinlist = ['liqu','manua']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Penetration Testing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['risk']
allinlist = ['it']
notinlist = ['cred','commo','miti']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['IT Risk'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['manag']
allinlist = ['it','ment']
notinlist = ['siti','dit','rit','uit','lit','fit','nit','pit','git','xit','sit','tit','cit','hit','vit','kit','inter','asset','deli','infra','oper','prod','prog','proj','erv','team','vend']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['IT Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['manag']
allinlist = ['identity']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Identity Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['techni']
allinlist = ['servic',]
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Technical Services'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['techni']
allinlist = ['analy',]
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Technical Analysis'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['deve']
allinlist = ['softw',]
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Software Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['engi']
allinlist = ['syst',]
notinlist = ['air']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Systems Engineering'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['bene']
allinlist = ['comp',]
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Benefits & Compensation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mana']
allinlist = ['perfo',]
notinlist = ['manager']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Performance Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['hris']
allinlist = []
notinlist = ['ghris']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Human Resources Information System (HRIS)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['safe']
allinlist = ['compl']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Safety Compliance'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['adp']
allinlist = ['adp']
notinlist = ['tooth','staa']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['ADP Payroll System'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['producti']
allinlist = ['mana']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Production Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['lean']
allinlist = ['manu']
notinlist = ['clea']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Lean Manufacturing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['machi']
allinlist = ['oper']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Machine Operation'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sap']
allinlist = ['prod']
notinlist = ['cost']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SAP Production Planning (SAP PP)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['supp']
allinlist = ['prod','tion']
notinlist = ['phone']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Production Support'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cod']
allinlist = ['medi']
notinlist = ['media']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Medical Coding'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['emr']
allinlist = []
notinlist = ['spark','aws','rush']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Electronic Medical Records (EMR)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['medi']
allinlist = ['reco']
notinlist = ['medium','mmedi','media']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Medical Records'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['acc']
allinlist = ['recei']
notinlist = ['medium','mmedi','media']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Accounts Receivable'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['radiology']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Radiology'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['billing']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Billing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['claim']
allinlist = ['hand']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Claim Handling'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['interven']
allinlist = ['rad']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Interventional Radiology'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['eval']
allinlist = ['mana']
notinlist = ['recr']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Evaluation and Management Auditing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['icd']
allinlist = []
notinlist = ['gicd','cicd','hicd']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['International Statistical Classifications of Diseases - 10 (ICD - 10) Coding'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['cpt']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Current Procedural Terminology (CPT) Coding'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['bill']
allinlist = ['coll']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Bill Collections'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['surg']
allinlist = ['code']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Surgical coding'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['medi']
allinlist = ['care']
notinlist = ['thcare','criti']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Medicare'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['auditing']
allinlist = []
notinlist = ['tl']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Auditing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['orthopedi']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Orthopedics'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['branding']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Branding'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['event']
allinlist = ['manag']
notinlist = ['preven']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Event Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['web']
allinlist = ['marke']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Web Marketing'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['manag']
allinlist = ['marketi']
notinlist = ['place']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Marketing Management'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['media','publ']
allinlist = ['rela']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Media / Public Relations'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['adver']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Advertising'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['event']
allinlist = ['plan']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Event Planning'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['soli']
allinlist = ['wor']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['SolidWorks'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['elem']
allinlist = ['finit']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Finite Element Analysis (FEA)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['fea']
allinlist = []
notinlist = ['feat','feas']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Finite Element Analysis (FEA)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['graph']
allinlist = ['uni']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['NX Unigraphics'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['matlab']
allinlist = ['lab']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Matlab'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['verilog']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Verilog'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['catia']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['CATIA'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['revit']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Autodesk Revit MEP'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ansys']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['ANSYS Simulation Software'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['inven']
allinlist = []
notinlist = ['tory','ries']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Autodesk Inventor'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ios']
allinlist = []
notinlist = ['bios','tios','xios','lios','gios','dios','rios']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['iOS'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['swift']
allinlist = []
notinlist = ['plan']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Swift Programming Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['obje']
allinlist = ['c','tive']
notinlist = ['acqu','pega','lead']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Objective-C'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['kotlin']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Kotlin Programming Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['reac']
allinlist = ['nati']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['React Native'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['flutter']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Flutter'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['mobil']
allinlist = ['develop']
notinlist = ['appi','apple','auto',]
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Mobile applications development'] = apl
#------------------------------------------------------------------------------------------------------
anyinlist = ['dart']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Dart Programming Language'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['ionic']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Ionic'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['androi']
allinlist = []
notinlist = ['rx']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Android Operating System Development'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['xamarin']
allinlist = []
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Xamarin'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['sdk']
allinlist = []
notinlist = ['agile','aws','c4c','crm','face','flut','goog','image','ios','java','native','integrations','sdks']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Android Software Development Kit (SDK)'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['payroll',]
allinlist = ['admi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Payroll Administration'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['bene',]
allinlist = ['admi']
notinlist = []
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['Benefits Administration'] = apl 
#------------------------------------------------------------------------------------------------------
anyinlist = ['adp',]
allinlist = []
notinlist = ['head','aadp']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
skills_edited['ADP Human Resources System'] = apl 
#------------------------------------------------------------------------------------------------------


In [83]:
anyinlist = ['human','resources']
allinlist = ['system']
notinlist = ['head','aadp']
# notinlist.extend( list(skills_edited['C Programming Language']))
# notinlist.remove('c')

apl = set()
for skill in skills:
    if isinstance(skill,str):    
        if any(word in skill for word in anyinlist) and all(word not in skill for word in notinlist) and all(word in skill for word in allinlist):
            apl.add(skill)
apl
# skills_edited['Benefits Administration'] = apl 

{'human resources management systems & applicant tracking systems.'}

In [84]:
l = '''
Human Resources (HR)
Employee Relations
Recruiting
Human Resources Information System (HRIS)
Benefits Administration
Benefits & Compensation
Full-Cycle Recruiting
Payroll Administration
Performance Management
Leave of Absence Administration
Training
Legal Compliance
Hiring
Project Management
ADP Payroll System
Microsoft Office
ADP Human Resources System
Microsoft Excel
Microsoft Word
Employee Engagement
'''
l = l.strip().split('\n')
l = "{'"+"','".join(l)+"'}"
l

"{'Human Resources (HR)','Employee Relations','Recruiting','Human Resources Information System (HRIS)','Benefits Administration','Benefits & Compensation','Full-Cycle Recruiting','Payroll Administration','Performance Management','Leave of Absence Administration','Training','Legal Compliance','Hiring','Project Management','ADP Payroll System','Microsoft Office','ADP Human Resources System','Microsoft Excel','Microsoft Word','Employee Engagement'}"

In [85]:
l = {'Human Resources (HR)','Employee Relations','Recruiting','Human Resources Information System (HRIS)','Benefits Administration','Benefits & Compensation','Full-Cycle Recruiting','Payroll Administration','Performance Management','Leave of Absence Administration','Training','Legal Compliance','Hiring','Project Management','ADP Payroll System','Microsoft Office','ADP Human Resources System','Microsoft Excel','Microsoft Word','Employee Engagement'}
c = 0
for i in l:
    if all(word.lower() != i.lower() for word in skills_edited):
        c += 1
        print(c,'-', i)

1 - Microsoft Word
2 - Employee Engagement
3 - Leave of Absence Administration


In [86]:
c = 0
for i in skills_edited:
    if 'jav' in i.lower():
        c += 1
        print(c,'-', i)

1 - Java
2 - Java Script


In [87]:
len(skills_edited)

278

In [88]:
reverse_mapping = {title.lower(): category for category, titles in skills_edited.items() for title in titles}
def map_title(value):
    if value is None:
        return None
    if isinstance(value, float):
        return None 
    else:
        if (len(value)>35) or value == '':
            return None 
        if (len(value)<2) and (value != 'c'):
            return None 
        lower_value = value.lower()
        return reverse_mapping.get(lower_value)

for i in skilcol:
    df[i] = df[i].apply(map_title)
    df[i] = df[i].apply(lambda x: None if len(str(x))<2 else x)
    df[i] = df[i].apply(lambda x: None if str(x).isdigit() else x)

In [89]:
df['combined_skills'] = df[skilcol].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)

In [90]:
skills_filter = ['Business Development','Client Interaction','New Business Development','Lead Generation','Business to Business (B2B) Sales','Oral / Verbal Communication','Market Research','Sales','Project Management','Customer Relationship Management (CRM)','Account Management','Strategic Partners','Strategic Sales','Negotiation','Inside Sales','Technical Sales','Marketing Communications','Strategic Marketing','Sales Management','Product Development']

any_title_filter = ['busine','sale','inside','outside']
all_title_filter = ['exec']
not_title_filter = ['java']

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Business Development Executive (BDE)'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Sales','Customer Relationship Management (CRM)','Business Strategy','Account Management','Sales Management','Project Management','Business Development','Business to Business (B2B) Sales','Strategic Planning','People Management','Contract Negotiation','Market Research','Strategic Sales','Technical Sales','Marketing Communications','Strategic Partners','Marketing Management','Strategic Marketing','Microsoft Office']

any_title_filter = ['busine','sale','dev']
all_title_filter = ['manag']
not_title_filter = ['java']

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Business Development Manager (BDM)'
#------------------------------------------------------------------------------------------------------
skills_filter = ['JavaScript','React.js','Node.js','SQL','Java','Angular','.NET','Python','PHP','C# Programming Language','Web Development','Amazon Web Services (AWS)','Vue.js','Typescript','MySQL','Spring','Django','Angular.js','Ruby on Rails','ASP.NET Framework']

any_title_filter = ['stac','dev']
all_title_filter = ['full']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Full Stack Developement'
#------------------------------------------------------------------------------------------------------
skills_filter = ['SQL','JavaScript','C# Programming Language','Java','.NET','Python','React.js','Angular','C++ Programming Language','Node.js','PHP','Amazon Web Services (AWS)','Typescript','HTML','Spring','MySQL','Linux','C Programming Language','Ruby on Rails','Go (Golang) Programming Language']

any_title_filter = ['dev']
all_title_filter = ['soft']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Software Development'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Sales','Microsoft Office','Customer Service','Outbound Calls']

any_title_filter = ['tele']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Tele Sales'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Recruiting','Full-Cycle Recruiting','Applicant Tracking System (ATS) Software','Sourcing','Oral / Verbal Communication','Hiring','Human Resources (HR)','Customer Service','Client Interaction','Sales','People Management','Customer Relationship Management (CRM)','Training','Negotiation','Social Media Marketing','Employee Relations','Microsoft Office','Scheduling','Administration','Business Development']

any_title_filter = ['recru','tale']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Recruitment & Talent Acquisition'
#------------------------------------------------------------------------------------------------------
skills_filter = ['JavaScript','Node.js','Python','SQL','MySQL','Java','PHP','Amazon Web Services (AWS)','PostgreSQL','.NET','C# Programming Language','MongoDB','Microservices','Go (Golang) Programming Language','Django','Ruby on Rails','Typescript','Linux','Express (Node.js)','Spring']

any_title_filter = ['dev','end']
all_title_filter = ['back']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Backend Developer'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Field Sales','Business to Business (B2B) Sales','Sales','Account Management','Marketing Communications','Customer Relationship Management (CRM)','Strategic Sales','Technical Sales','Customer Relations','Sales Management','Business Development','Customer Service','Project Management','Training','Negotiation','Presentations','New Business Development','Oral / Verbal Communication','Microsoft Excel','Operations Management']

any_title_filter = ['exec','sales']
all_title_filter = ['field']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Field Sales Executive'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Information Technology (IT) Support','Troubleshooting','Help Desk / Desktop Support (Tier 2)','System Administration','Help Desk / Desktop Support','Computer Hardware Technician','Customer Service','Network Management / Administration','Microsoft Active Directory','Help Desk / Desktop Support (Tier 3)','Network Support','Systems Troubleshooting','Software Installation & Upgrade','Project Management','Microsoft Office','PC Support','Software Knowledge','Windows Support','Microsoft Exchange','Windows Server 2012 R2']

any_title_filter = ['info','tech','sup']
all_title_filter = ['it']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'IT Support'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Sales Management','New Business Development','Strategic Sales','Customer Relationship Management (CRM)','Business Development','Account Management','Field Sales','Sales','Business to Business (B2B) Sales','Customer Service','Technical Sales','Negotiation','Customer Relations','Contract Negotiation','Inside Sales','Salesforce','Oral / Verbal Communication','Marketing Communications','Channel Sales','Project Management']

any_title_filter = ['sale']
all_title_filter = ['dire']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Direct Sales Executive'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Customer Service','Sales Administration','Customer Relationship Management (CRM)','Microsoft Excel','Microsoft Office','Salesforce','Customer Relations','Data Entry','Database Management & Reporting','Process Improvement','Account Management','Data Analysis','Oral / Verbal Communication','Inside Sales','Project Management','Proposal Writing','Pricing','Sales','Training','Presentations']

any_title_filter = ['sale']
all_title_filter = ['supp','oper']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Sales Support & Operations'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Financial Analysis','Microsoft Excel','Financial Modeling','Operations Management','Risk Management / Risk Control','Mergers and Acquisitions','Banking','Project Management','Investment Management','Data Analysis','Reconciliation','Financial Reporting','Customer Relationship Management (CRM)','Customer Service','Microsoft Office','Investor Relations','Python','Business Intelligence','PL/SQL','Shell Scripting']

any_title_filter = ['bank','trad','inves']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'BFSI, Investments & Trading'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Field Sales','Sales','Business to Business (B2B) Sales','Account Management','Marketing Communications','Customer Relationship Management (CRM)','Strategic Sales','Technical Sales','Customer Relations','Sales Management','Business Development','Customer Service','Project Management','Training','Negotiation','Presentations','New Business Development','Oral / Verbal Communication','Microsoft Excel','Operations Management']

any_title_filter = ['area']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Area Sales Manager (B2B)'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Customer Relationship Management (CRM)','Sales','Customer Relations','Client Interaction','Account Management','Financial Analysis','Oral / Verbal Communication','Business Development','Banking','Project Management','Commercial Loans','Financial Advisor','Customer Service','Sales Management','Marketing Communications','Negotiation','Salesforce','Loan Underwriting','Program Management','Human Resources (HR)']

any_title_filter = ['rela']
all_title_filter = ['mana']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Relationship Manager'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Customer Relationship Management (CRM)','Sales','Customer Relations','Client Interaction','Account Management','Financial Analysis','Oral / Verbal Communication','Business Development','Banking','Project Management','Commercial Loans','Financial Advisor','Customer Service','Sales Management','Marketing Communications','Negotiation','Salesforce','Loan Underwriting','Program Management','Human Resources (HR)']

any_title_filter = ['mark']
all_title_filter = ['digi']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Digital Marketing'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Customer Service','Retail','Cash Handling','Merchandising','Customer Relations','Client Interaction','Oral / Verbal Communication','Sales','Inventory Management','Organizing','Training','Leadership','Forklift Certified','Spanish Language','Maintenance','Sales Management','Customer Relationship Management (CRM)','Design','Inside Sales','Microsoft Office']

any_title_filter = ['reta']
all_title_filter = ['sal']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Retail Sales'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Cyber Security','Security Risk Management','Security Policies and Procedures','IT Security & Infrastructure','Security Testing and Auditing','Risk Management / Risk Control','Network Security Management','ISO 27001','Computer Security','Team Management','Security Information and Event Management (SIEM)','Vulnerability Assessment','ISO/IEC 27001','Project Management','Regulatory Compliance','Amazon Web Services (AWS)','Penetration Testing','IT Risk','IT Management','Identity Management']

any_title_filter = ['secu']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'IT & Information Security'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Technical Services','Technical Analysis','Systems Troubleshooting','Project Management','Information Technology (IT) Support','Software Development','Engineering Design','Data Analysis','Electronic Troubleshooting','Microsoft Office','Training','Microsoft Azure','Python','Java','Product Development','.NET','Development Operations (DevOps)','Systems Engineering','C Programming Language','Technical Writing']

any_title_filter = ['tech']
all_title_filter = ['lead']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Tech Lead'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Human Resources (HR)','Employee Relations','Benefits & Compensation','Performance Management','Recruiting','Payroll Administration','Full-Cycle Recruiting','Generalist Duties','People Management','Human Resources Information System (HRIS)','Strategic Planning','Organizational Development','Legal Compliance','Benefits Administration','Training','Labor Relations','Hiring','Safety Compliance','ADP Payroll System','Microsoft Excel']

any_title_filter = ['hr','human']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Human Resources'
#------------------------------------------------------------------------------------------------------
skills_filter = ['Production Management','People Management','Problem Solving','Leadership','Team Leadership','Lean Manufacturing','Quality Assurance / Quality Control','Microsoft Office','Training','Process Improvement','Machine Operation','Scheduling','Safety Compliance','Operations Management','Microsoft Excel','SAP Production Planning (SAP PP)','Oral / Verbal Communication','Project Management','Maintenance','Production Support']

any_title_filter = ['prod']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Production & Manufacturing'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Medical Coding','International Statistical Classifications of Diseases - 10 (ICD - 10) Coding','Current Procedural Terminology (CPT) Coding','Electronic Medical Records (EMR)','Billing','Surgical coding','Insurance','Medical Terminology','Medical Records','Epic Health Information Management System','Emergency Room (ER)','Auditing','Interventional Radiology','Quality Assurance / Quality Control','Orthopedics','Evaluation and Management Auditing','Microsoft Office','Emergency/Trauma','Radiology','Medicaid & Medicare Billing','Billing','Medicaid & Medicare Billing','Insurance','Accounts Receivable','Data Entry','Medical Coding','Bill Collections','Claim Handling','Electronic Medical Records (EMR)','International Statistical Classifications of Diseases - 10 (ICD - 10) Coding','Medicaid Billing','Medical Terminology','Epic Hospital Billing','Office Administration','Medicare','Medical Credentialing','Current Procedural Terminology (CPT) Coding','Microsoft Excel','Team Leadership','Microsoft Office'}

any_title_filter = ['bill']
all_title_filter = ['medi']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Medical Biller'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Medical Coding','International Statistical Classifications of Diseases - 10 (ICD - 10) Coding','Current Procedural Terminology (CPT) Coding','Electronic Medical Records (EMR)','Billing','Surgical coding','Insurance','Medical Terminology','Medical Records','Epic Health Information Management System','Emergency Room (ER)','Auditing','Interventional Radiology','Quality Assurance / Quality Control','Orthopedics','Evaluation and Management Auditing','Microsoft Office','Emergency/Trauma','Radiology','Medicaid & Medicare Billing','Billing','Medicaid & Medicare Billing','Insurance','Accounts Receivable','Data Entry','Medical Coding','Bill Collections','Claim Handling','Electronic Medical Records (EMR)','International Statistical Classifications of Diseases - 10 (ICD - 10) Coding','Medicaid Billing','Medical Terminology','Epic Hospital Billing','Office Administration','Medicare','Medical Credentialing','Current Procedural Terminology (CPT) Coding','Microsoft Excel','Team Leadership','Microsoft Office'}

any_title_filter = ['code']
all_title_filter = ['medi']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Medical Coder'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Marketing Management','Strategic Marketing','Marketing Communications','Social Media Marketing','Project Management','Digital Marketing','Email Marketing','Online Marketing','Graphic Design','Content Management','Branding','Advertising','Event Management','Data Analysis','Search Engine Optimization (SEO)','Event Planning','Market Research','Media / Public Relations','Marketing Administration','Web Marketing'}

any_title_filter = ['market','busi']
all_title_filter = ['manag']
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Marketing Manager'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Engineering Design','Computer Aided Design (CAD)','SolidWorks','Product Development','Autodesk AutoCAD','Microsoft Office','Microsoft Excel','Project Management','CATIA','Design','Finite Element Analysis (FEA)','Autodesk Inventor','Pro/Engineer','NX Unigraphics','Computer Aided Drafting & Design (CADD)','Autodesk AutoCAD Civil 3d','ANSYS Simulation Software','Matlab','Autodesk Revit MEP','verilog'}

any_title_filter = ['desi']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Design Engineer'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Sales Management','Team Management','Customer Relationship Management (CRM)','Team Leadership','Operations Management','Field Sales','Strategic Sales','Business Development','Sales','Technical Sales','Insurance','Customer Relations','Business to Business (B2B) Sales','Strategic Project Management','Customer Service','Budget Management','People Management','Performance Management','Project Management','Leadership'}

any_title_filter = ['branc','b2c']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Branch Sales Manager'
#------------------------------------------------------------------------------------------------------
skills_filter = {'Mobile applications development','iOS','Android Software Development Kit (SDK)','flutter','React Native','JavaScript','Java','Xamarin','Swift Programming Language','Kotlin Programming Language','React.js','Dart Programming Language','C# Programming Language','PL/SQL','Ionic','Angular','Typescript','.NET','Objective-C','Android Operating System Development'}

any_title_filter = ['app','mobi']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Mobile / App Developer'
#------------------------------------------------------------------------------------------------------

In [91]:
len(df['mapped_job_title'].unique())

41

In [92]:
l = '''
'''
l = l.strip().split('\n')
l = "{'"+"','".join(l)+"'}"
l

"{''}"

In [93]:
l = {'business development executive (bde)','business development manager (bdm)','full stack developer','software development - other','telesales','field sales executive','recruitment &amp; talent acquisition - other','back end developer','it support - other','direct sales executive','sales support &amp; operations - other','front end developer','relationship manager','digital marketing - other','bfsi, investments &amp; trading - other','area sales manager (b2b)','enterprise &amp; b2b sales - other','retail sales','it &amp; information security - other','human resources - other','technical lead','medical biller / coder','hr generalist','healthcare &amp; life sciences - other','design engineer','it recruiter','non it recruiter','hr operations - other','branch sales executive','production &amp; manufacturing - other','retail banking sales','mobile / app developer','marketing manager','branch sales manager (b2c)','key account manager','marketing - other','data engineer','it infrastructure services - other','regional sales manager (b2c)','graphic designer','functional consultant','area sales manager (b2c)','technical consultant','bd / pre sales - other','marketing and communication - other','social media marketing','wealth manager','regional sales manager (b2b)','enterprise sales manager','business analyst','technical architect','hardware and networks - other','automation test engineer','sales head (b2b)','insurance sales / bd manager','territory sales manager (b2b)','lead generation / qualification'}
malfun = ['full stack developer','front end developer','back end developer','social media marketing','marketing','marketing and communication']
for i in l:
    i = i.replace('amp;','').replace(' - other','')
    if all(word.lower() not in i for word in df['mapped_job_title'].unique()) and i not in malfun:
        print(i)

wealth manager
healthcare & life sciences
key account manager
functional consultant
technical lead
hr operations
graphic designer
hr generalist
lead generation / qualification
technical architect
technical consultant
hardware and networks
it infrastructure services


In [94]:
skills_filter = {'Mobile applications development','iOS','Android Software Development Kit (SDK)','flutter','React Native','JavaScript','Java','Xamarin','Swift Programming Language','Kotlin Programming Language','React.js','Dart Programming Language','C# Programming Language','PL/SQL','Ionic','Angular','Typescript','.NET','Objective-C','Android Operating System Development'}

any_title_filter = ['app','mobi']
all_title_filter = []
not_title_filter = []

skills_mask = df['combined_skills'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in skills_filter))
job_title_mask_any = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in any_title_filter))
job_title_mask_all = df['job_title'].apply(lambda x: isinstance(x, str) and all(keyword.lower() in x.lower() for keyword in all_title_filter))
job_title_mask_not = df['job_title'].apply(lambda x: isinstance(x, str) and any(keyword.lower() in x.lower() for keyword in not_title_filter))

# df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not, 'mapped_job_title'] = 'Mobile / App Developer'
# for i in df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not]['job_title']:
#     print(i)
df.loc[skills_mask & job_title_mask_any & job_title_mask_all & ~job_title_mask_not]

,job_title,company_name,experience,salary,location,description,posted_on,current_date,skill1,skill2,...,industry type,department,employment type,role category,job_description,mapped_average_sal,avg_experience,mapped_job_title,mapped_job_title_1,combined_skills
24,Lead Software Engineer/Staff software Backend ...,amolitalents,7-12 Yrs,25-40 Lacs PA,Bangalore/Bengaluru,. Bachelors degree in CS / EE or equivalent in...,1 Day Ago,2023-08-22,Kubernetes,Node.js,...,Nan,Nan,Nan,Nan,Nan,32.500,9.5,Mobile / App Developer,Lead Engineer Staff Software Backend Cloud Apps,"Kubernetes,Node.js,Java Script,Java Script"
149,Sr Mobile and Selenium Automation Tester,tech mahindra,5-10 Yrs,12-20 Lacs PA,"Hybrid - Hyderabad/ Secunderabad, Telangana, B...",SDET - UI AutomationOverall 7-12 years of expe...,12 Days Ago,2023-08-22,Java Script,Automation Scripting,...,Nan,Nan,Nan,Nan,Nan,16.000,7.5,Mobile / App Developer,Sr Mobile Selenium Automation Tester,"Java Script,Automation Scripting,Automation Sc..."
345,Mobile - Engineering Manager,anko,12-17 Yrs,40-50 Lacs PA,Hybrid - Bangalore/Bengaluru,Anko is an innovation and technology services ...,13 Days Ago,2023-08-22,None,Flutter,...,Nan,Nan,Nan,Nan,Nan,45.000,14.5,Mobile / App Developer,Mobile Engineering Manager,"Flutter,iOS,Node.js,React.js"
528,Team Lead - Desktop Application in a leading f...,fortune human capital (p) ltd,6-11 Yrs,12-22 Lacs PA,"Bangalore/Bengaluru, Noida, Mumbai","Experience with debugging, performance profili...",7 Days Ago,2023-08-22,C# Programming Language,None,...,Nan,Nan,Nan,Nan,Nan,17.000,8.5,Mobile / App Developer,Team Lead Desktop Application Financial,"C# Programming Language,SQL"
552,Senior Director - Mobility,lemon tree staffing,14-20 Yrs,50-60 Lacs PA,Bangalore/Bengaluru,Previous mobile app. development experience us...,8 Days Ago,2023-08-22,None,Microservices,...,Nan,Nan,Nan,Nan,Nan,55.000,17,Mobile / App Developer,Senior Director Mobility,"Microservices,React.js"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52041,Senior Application Engineer,applied information sciences (ais),7-12 Yrs,20-35 Lacs PA,Hybrid - Hyderabad/Secunderabad(Kondapur),Role : Senior Application Engineer Requirement...,1 Day Ago,2023-11-15,Java Script,None,...,it services & consulting,engineering - software & qa,"full time, permanent",software development,Role : Senior Application Engineer\nRequiremen...,27.500,9.5,Mobile / App Developer,Senior Application Engineer,"Java Script,Microsoft Azure,Microsoft Azure"
52073,Urgent Hiring||Application Monitoring Engineer...,parth associates,3-6 Yrs,10-15 Lacs PA,"Pune, Maharashtra",2+ year experience in cloud monitoring such as...,1 Day Ago,2023-11-15,Java Script,None,...,it services & consulting,it & information security,"full time, permanent",it security,2+ year experience in Application Monitoring t...,12.500,4.5,Mobile / App Developer,Application Monitoring Engineer,"Java Script,Linux,Microsoft Azure,Python,.NET,..."
52077,Android Application Developer,pronexus consulting,2-5 Yrs,6.5-10 Lacs PA,Remote,Explore and implement innovative features and ...,1 Day Ago,2023-11-15,Android Operating System Development,Android Operating System Development,...,it services & consulting,engineering - software & qa,"full time, freelance/homebased",software development,Role & responsibilities\nMobile App Developmen...,8.250,3.5,Mobile / App Developer,Android Application Developer,"Android Operating System Development,Android O..."
52393,Mobile Software Engineer,biologique croissance technologies,2-5 Yrs,4.5-8.5 Lacs PA,Remote,Role & responsibilities 1. Develop and maintai...,1 Day Ago,2023-11-15,Mobile applications development,Mobile applications development,...,it services & consulting,engineering - software & qa,"full time, freelance/homebased",software development,Role & responsibilities\n1. Develop and mainta...,6.500,3.5,Mobile / App Developer,Mobile Software Engineer,"Mobile applications development,Mobile applica..."


In [95]:
skillvalcount = {}

def valcount(lis):
    for i in lis:
        if i in skills_edited:
            if i in skillvalcount:
                skillvalcount[i] += 1
            else:
                skillvalcount[i] = 1

for index, val in df[skilcol].iterrows():
    val = val.to_list()
    valcount(val)

In [96]:
skillvalcountdf = pd.DataFrame.from_dict(skillvalcount, orient='index',columns=['count'])
skillvalcountdf.index.name = 'skills from payscale'
skillvalcountdf.to_excel('mapped_pay_scale_skills.xlsx')

In [97]:
pd.DataFrame.from_dict(skills_edited, orient='index').T.to_excel('mapped_pay_scale_skills.xlsx',index=False)

In [98]:
# for k,v in skills_edited.items():
#     skills_edited[k]=len(v)

In [99]:
# p = pd.DataFrame.from_dict(skills_edited, orient='index',columns=['counts'])
# p.index.name = 'skills'
# p.to_excel('payscale_skills_mapped_counts.xlsx')

In [100]:
df['combined_skills'] = df[skilcol].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)

In [101]:
df['mapped_job_title'].unique()

array(['Engineer', 'unclassified', 'Voice Process', 'Customer Support',
       'Data Scientist', 'Tele Sales',
       'Business Development Manager (BDM)', 'Area Sales Manager (B2B)',
       'Retail Sales', 'Sales', 'Business Development Executive (BDE)',
       'Business Development', 'Mobile / App Developer',
       'Business Analyst', 'Backend Developer', 'Software Development',
       'Digital Marketing', 'Data Engineer', 'Human Resources',
       'Marketing Manager', 'Design Engineer', 'Full Stack Developement',
       'Recruitment & Talent Acquisition', 'Lead/Management', 'Tech Lead',
       'IT Support', 'BFSI, Investments & Trading',
       'Production & Manufacturing', 'Recruiter', 'Project Management',
       'Devops', 'Relationship Manager', 'Product Management',
       'Field Sales Executive', 'Branch Sales Manager',
       'IT & Information Security', 'Counsellor', 'Medical Biller',
       'Medical Coder', 'Direct Sales Executive',
       'Sales Support & Operations'], dty

In [102]:
renjidict = {}

for i in df['mapped_job_title'].unique():
    filtdf = df[df['mapped_job_title'] == i][skilcol]
    for index, j in filtdf.iterrows():        
        for k in j:
            if (k != '') and isinstance(k, str):
                if not (len(k)<2):        
                    if i in renjidict:
                        renjidict[i].add(k)
                    else:
                        renjidict[i] = set(k)

In [103]:
for key, value_set in renjidict.items():
    filtered_set = {word for word in value_set if len(word) > 2 and not word.isdigit()}
    renjidict[key] = filtered_set

In [104]:
allskills = set()
for col in renjidict:
    allskills.update(renjidict[col])
len(allskills)

274

In [105]:
value_counts = {}
for col in renjidict:
    value_counts[col] = len(renjidict[col])
converted = pd.DataFrame.from_dict(value_counts, orient='index')
converted.to_excel('skill_value_counts.xlsx')

In [106]:
converted

,0
Engineer,201
unclassified,253
Voice Process,70
Customer Support,110
Data Scientist,165
Tele Sales,44
Business Development Manager (BDM),119
Area Sales Manager (B2B),50
Retail Sales,33
Sales,170


In [107]:
sum = 0
for i in value_counts:
    sum += value_counts[i] 
print(sum)

3547


# value counts of payscale skills

In [108]:
renjidictvc = {} 

for i in df['mapped_job_title'].unique():
    filtdf = df[df['mapped_job_title'] == i][skilcol]
    skill_counts = {}
    for index, j in filtdf.iterrows():        
        for k in j:
            if (k != '') and isinstance(k, str) and k != 'nan':
                if k in skill_counts:
                    skill_counts[k] += 1
                else:
                    skill_counts[k] = 1
    renjidictvc[i] = skill_counts

print(renjidictvc)


{'Engineer': {'Terraform': 102, 'Development Operations (DevOps)': 326, 'Kubernetes': 136, 'Amazon Web Services (AWS)': 402, 'Java Script': 366, 'React.js': 69, 'Spring Boot': 31, 'Microservices': 51, 'Agile Software Development': 38, 'Spring': 31, 'Go (Golang) Programming Language': 23, 'Software Development': 67, 'PHP': 44, 'MySQL': 41, 'Oral / Verbal Communication': 169, 'Information Technology (IT) Support': 179, 'Technical Analysis': 26, 'Angular': 58, 'Linux': 177, 'Typescript': 20, 'Python': 412, 'C++ Programming Language': 86, 'Microsoft Azure': 549, 'Automation Scripting': 279, 'Maintenance': 547, 'Scala': 44, 'SQL': 433, 'Spark': 184, 'Strategic Marketing': 11, 'Sales': 857, 'Business Development': 83, 'Market Research': 6, 'Documentation': 29, 'Presentations': 30, 'Autodesk AutoCAD': 100, 'CATIA': 17, 'Project Management': 42, 'Troubleshooting': 277, 'Google Cloud Platform (GCP)': 111, 'Data Modeling': 41, 'Shell Scripting': 50, 'Java': 108, 'ASP.NET Framework': 66, 'Autodes

In [109]:
pd.DataFrame.from_dict(renjidictvc, orient='index').T.to_excel('payscale_skill_value_counts.xlsx')

In [110]:
payscale_skilldf = pd.read_excel('payscale_skill_value_counts.xlsx')

In [111]:
columns_to_sum = payscale_skilldf.columns.drop(['unclassified','Unnamed: 0'])
payscale_skilldf['count'] = payscale_skilldf[columns_to_sum].sum(axis=1)

In [112]:
payscale_skilldf = payscale_skilldf.sort_values(by='count', ascending=False)

In [113]:
# payscale_skilldf = payscale_skilldf[~(payscale_skilldf['Unnamed: 0'].str.islower()) & (payscale_skilldf['count'] < 500)]

In [114]:
roles = list(payscale_skilldf.columns)
roles.remove('Unnamed: 0')
roles.remove('count')
roles.remove('unclassified')

In [115]:
topskillsdf = pd.DataFrame()

In [116]:
for i in range(len(roles)):
    temptop = payscale_skilldf.sort_values(by=roles[i], ascending=False)
    temptop = temptop[['Unnamed: 0', roles[i]]]
    temptop.rename(columns={'Unnamed: 0': roles[i], roles[i]: 'count'}, inplace=True)
    topskillsdf = pd.concat([topskillsdf, temptop[[roles[i], 'count']].head(20)], ignore_index=True)

In [117]:
top_skills_data = {}
for i in range(len(roles)):
    temptop = payscale_skilldf.sort_values(by=roles[i], ascending=False)
    top_skills_data[roles[i]] = temptop['Unnamed: 0'].head(20).tolist()
    top_skills_data[f'{roles[i]}_count'] = temptop[roles[i]].head(20).tolist()

# Create a DataFrame from the dictionary
topskillsdf = pd.DataFrame(top_skills_data)

In [118]:
topskillsdf

,Engineer,Engineer_count,Sales,Sales_count,Software Development,Software Development_count,Digital Marketing,Digital Marketing_count,Lead/Management,Lead/Management_count,...,Branch Sales Manager,Branch Sales Manager_count,Counsellor,Counsellor_count,Medical Biller,Medical Biller_count,Medical Coder,Medical Coder_count,Sales Support & Operations,Sales Support & Operations_count
0,Sales,857.0,Sales,5213.0,Java Script,1784.0,Sales,849.0,Sales,1535.0,...,Sales,695.0,Sales,607.0,Billing,43.0,Medical Coding,191.0,Sales,8.0
1,Microsoft Azure,549.0,Oral / Verbal Communication,520.0,.NET,1045.0,Digital Marketing,500.0,Insurance,538.0,...,Insurance,310.0,Oral / Verbal Communication,149.0,Accounts Receivable,5.0,International Statistical Classifications of D...,36.0,Sales Administration,2.0
2,Maintenance,547.0,Business Development,472.0,SQL,1007.0,Search Engine Optimization (SEO),372.0,Oral / Verbal Communication,253.0,...,Banking,155.0,Inside Sales,109.0,Oral / Verbal Communication,3.0,Current Procedural Terminology (CPT) Coding,22.0,Administration,1.0
3,SQL,433.0,Insurance,434.0,Java,708.0,Social Media Marketing,360.0,Business Development,251.0,...,Retail,97.0,Business Development,86.0,Insurance,3.0,Billing,7.0,Business Development,NaN
4,Python,412.0,Inside Sales,331.0,PHP,652.0,Oral / Verbal Communication,354.0,Retail,212.0,...,Business Development,78.0,Administration,31.0,Business Development,2.0,Radiology,7.0,Recruiting,NaN
5,Amazon Web Services (AWS),402.0,Lead Generation,261.0,HTML,612.0,Advertising,256.0,Banking,208.0,...,Channel Sales,45.0,Lead Generation,26.0,Customer Service,2.0,Hiring,6.0,Oral / Verbal Communication,NaN
6,Java Script,366.0,Business to Business (B2B) Sales,237.0,ASP.NET Framework,584.0,Google Ad Words,227.0,Account Management,205.0,...,Sales Management,44.0,Negotiation,17.0,Medical Coding,2.0,Oral / Verbal Communication,3.0,Human Resources (HR),NaN
7,Development Operations (DevOps),326.0,Customer Service,231.0,Spring Boot,529.0,Business Development,194.0,Maintenance,180.0,...,Team Management,32.0,Field Sales,16.0,Data Entry,2.0,Training,3.0,Insurance,NaN
8,Automation Scripting,279.0,Microsoft Excel,216.0,React.js,494.0,Branding,186.0,Leadership,159.0,...,Field Sales,26.0,Customer Service,13.0,Current Procedural Terminology (CPT) Coding,2.0,Medical Terminology,3.0,Java Script,NaN
9,Troubleshooting,277.0,Technical Sales,205.0,Angular,464.0,Google Analytics,152.0,Team Management,156.0,...,Strategic Sales,25.0,Channel Sales,10.0,Hiring,1.0,Presentations,2.0,Hiring,NaN


In [119]:
topskillsdf.to_excel('role_wise_top_skills .xlsx',index=False)

In [120]:
skillsdf = pd.DataFrame.from_dict(renjidict, orient='index').T
skillsdf.columns = skillsdf.columns.str.replace(r'\W+', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_').str.lower()
skillsdf.to_excel('skills_and_relevant_roles.xlsx',index =False)

In [121]:
topskillsdf = pd.read_excel('role_wise_top_skills .xlsx')
topskillsdf.columns = topskillsdf.columns.str.replace(r'\W+', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_').str.lower()

In [122]:
df.columns = df.columns.str.replace(r'\W+', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_').str.lower()
df.columns

Index(['job_title', 'company_name', 'experience', 'salary', 'location',
       'description', 'posted_on', 'current_date', 'skill1', 'skill2',
       'skill3', 'skill4', 'skill5', 'skill6', 'skill7', 'skill8', 'role',
       'industry_type', 'department', 'employment_type', 'role_category',
       'job_description', 'mapped_average_sal', 'avg_experience',
       'mapped_job_title', 'mapped_job_title_1', 'combined_skills'],
      dtype='object')

In [123]:
for i in topskillsdf.columns:
    if 'count' in i:
        del topskillsdf[i]

In [124]:
skillsdf.columns

Index(['engineer', 'unclassified', 'voice_process', 'customer_support',
       'data_scientist', 'tele_sales', 'business_development_manager_bdm',
       'area_sales_manager_b2b', 'retail_sales', 'sales',
       'business_development_executive_bde', 'business_development',
       'mobile_app_developer', 'business_analyst', 'backend_developer',
       'software_development', 'digital_marketing', 'data_engineer',
       'human_resources', 'marketing_manager', 'design_engineer',
       'full_stack_developement', 'recruitment_talent_acquisition',
       'lead_management', 'tech_lead', 'it_support',
       'bfsi_investments_trading', 'production_manufacturing', 'recruiter',
       'project_management', 'devops', 'relationship_manager',
       'product_management', 'field_sales_executive', 'branch_sales_manager',
       'it_information_security', 'counsellor', 'medical_biller',
       'medical_coder', 'direct_sales_executive', 'sales_support_operations'],
      dtype='object')

In [125]:
df.to_excel('salary_and_company_mapping.xlsx',index = False)

In [126]:
import mysql.connector
from sqlalchemy import create_engine
# Define your database connection parameters
host = "54.79.186.42"
user = "root"
password = "secondstorey"
database = "salary_app"

# Establish a connection to the database
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

# Check if the connection was successful
if connection.is_connected():
    print("Connected to MySQL database")

cursor = connection.cursor()
cursor.execute("SHOW DATABASES")

databases = cursor.fetchall()
for db in databases:
    print(db[0])
    
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
df.to_sql(name='naukri_extract', con=engine, if_exists='replace', index=False)
skillsdf.to_sql(name='skills_and_relevant_roles', con=engine, if_exists='replace', index=False)

Connected to MySQL database
gmat
information_schema
mysql
performance_schema
salary_app
secondstorey
sys


253

In [127]:
topskillsdf.to_sql(name='topskills', con=engine, if_exists='replace', index=False)

20

In [128]:
topskillsdf

,engineer,sales,software_development,digital_marketing,lead_management,tech_lead,devops,mobile_app_developer,business_analyst,backend_developer,...,area_sales_manager_b2b,retail_sales,marketing_manager,relationship_manager,field_sales_executive,branch_sales_manager,counsellor,medical_biller,medical_coder,sales_support_operations
0,Sales,Sales,Java Script,Sales,Sales,Java Script,Development Operations (DevOps),Android Operating System Development,Business Analysis,Java Script,...,Sales,Sales,Marketing Management,Sales,Sales,Sales,Sales,Billing,Medical Coding,Sales
1,Microsoft Azure,Oral / Verbal Communication,.NET,Digital Marketing,Insurance,Technical Analysis,Microsoft Azure,Java Script,SQL,Spring Boot,...,Sales Management,Retail,Digital Marketing,Insurance,Field Sales,Insurance,Oral / Verbal Communication,Accounts Receivable,International Statistical Classifications of D...,Sales Administration
2,Maintenance,Business Development,SQL,Search Engine Optimization (SEO),Oral / Verbal Communication,.NET,Amazon Web Services (AWS),iOS,Agile Software Development,Node.js,...,Business Development,Customer Service,Sales,Banking,Business Development,Banking,Inside Sales,Oral / Verbal Communication,Current Procedural Terminology (CPT) Coding,Administration
3,SQL,Insurance,Java,Social Media Marketing,Business Development,Spring Boot,Kubernetes,Mobile applications development,Oral / Verbal Communication,Java,...,Channel Sales,Business Development,Business Development,Business Development,Insurance,Retail,Business Development,Insurance,Billing,Business Development
4,Python,Inside Sales,PHP,Oral / Verbal Communication,Retail,Microservices,Terraform,Flutter,Sales,Microservices,...,Insurance,Channel Sales,Strategic Marketing,Retail,Business to Business (B2B) Sales,Business Development,Administration,Business Development,Radiology,Recruiting
5,Amazon Web Services (AWS),Lead Generation,HTML,Advertising,Banking,Java,Docker,.NET,Microsoft Excel,Amazon Web Services (AWS),...,Field Sales,Sales Management,Social Media Marketing,Field Sales,Oral / Verbal Communication,Channel Sales,Lead Generation,Customer Service,Hiring,Oral / Verbal Communication
6,Java Script,Business to Business (B2B) Sales,ASP.NET Framework,Google Ad Words,Account Management,Microsoft Azure,Jenkins,SQL,Requirements Analysis,Python,...,Business to Business (B2B) Sales,Field Sales,Search Engine Optimization (SEO),Oral / Verbal Communication,Banking,Sales Management,Negotiation,Medical Coding,Oral / Verbal Communication,Human Resources (HR)
7,Development Operations (DevOps),Customer Service,Spring Boot,Business Development,Maintenance,ASP.NET Framework,Python,React Native,Documentation,Spring,...,Retail,Commercial,Branding,Channel Sales,Presentations,Team Management,Field Sales,Data Entry,Training,Insurance
8,Automation Scripting,Microsoft Excel,React.js,Branding,Leadership,SQL,Java Script,Swift Programming Language,Data Analysis,.NET,...,Oral / Verbal Communication,Business to Business (B2B) Sales,Advertising,Customer Relations,Channel Sales,Field Sales,Customer Service,Current Procedural Terminology (CPT) Coding,Medical Terminology,Java Script
9,Troubleshooting,Technical Sales,Angular,Google Analytics,Team Management,Leadership,Automation Scripting,C# Programming Language,Technical Analysis,SQL,...,Team Management,Insurance,Oral / Verbal Communication,Business to Business (B2B) Sales,Lead Generation,Strategic Sales,Channel Sales,Hiring,Presentations,Hiring


In [129]:
# engine.dispose()

In [130]:
# cursor.execute("select * from secondstorey.tracking_data_3")
# rows = cursor.fetchall()
# columns = [desc[0] for desc in cursor.description]
# data_frame = pd.DataFrame(rows, columns=columns)
# data_frame

In [131]:
# import requests

# def get_ip_location(ip_address):
#     access_token = '255ebfcc46428e'  # Replace with your IPinfo API access token
#     api_url = f'https://ipinfo.io/{ip_address}/json?token={access_token}'
    
#     response = requests.get(api_url)
#     data = response.json()
    
#     return data

# def get_location(row):
#     ip_address = row
#     location_info = get_ip_location(ip_address)
#     return location_info

# data_frame['Location'] = data_frame['path'].apply(get_location)

# cities = []
# regions = []

# for entry in data_frame['Location']:
#     if 'city' in entry:
#         cities.append(entry['city'])
#     else:
#         cities.append(None) 
        
#     if 'region' in entry:
#         regions.append(entry['region'])
#     else:
#         regions.append(None)

# data_frame['City'] = cities
# data_frame['Region'] = regions


In [132]:
# data_frame.to_excel('tracking_data_3.xlsx',index=False)